# Extraindo Dados
___

Esse arquivo tem o objetivo de extrair dados do dataframe BSI e criar uma tabela que contenha os seguintes dados:

- o identificador do discente; 
- o sexo do discente;
- o ano em que o discente ingressou na instituição;
- o ano e período da última matrícula realizada pelo discente;
- total de sementres cursados pelo discente;
- um calcúlo de semestres cursados dividido pelo o número de semestre ideal do curso;
- a carga horária cumprida pelo discente;
- a carga horária cumprida pelo discente dividida pela carga horaria ideal;
- tempo relativo que o discente evadiu ou concluiu o curso;
- status atual do discente;
- número total de faltas de cada discente na disciplina que ele cursou;
- nota de cada discente nas disciplinas que ele cursou;
- média final de cada discente nas disciplinas que ele cursou;
- quantidade de vezes que o discente se matriculou nas disciplinas;
- quantidade de vezes que o discente se matriculou em cada disciplina;
- quantidade de vezes que o discente foi aprovado em cada disciplina;
- quantidade de vezes que o discente foi reprovado em cada disciplina.

Importando o Pandas e o Csv.

In [233]:
import pandas as pd
import numpy as np
import csv

Leitura do arquivo em csv e carregamento desses dados em um DataFrame usando o ponto e vírgula como separador.

In [234]:
#df_dados = pd.read_csv('dataframe-bsi-2009-2022.csv', sep=';')

In [235]:
#df_dados = pd.read_csv('dataframe_bsi_original.csv', sep=',')

In [236]:
df_dados = pd.read_csv('dataframe_ORIGINAL_DEZEMBRO.csv', sep=',')

In [237]:
df_dados

,discente,unidade,nota,reposicao,numero_total_faltas,sexo,ano_nascimento,estado_origem,cidade_origem,estado,municipio,media_final,descricao,ano,nome,ch_total,ano_ingresso,status
0,bad4e0ec61e635f382e7e32c32d5ad11,1.0,"0,0",False,44.0,F,1991,Rio Grande do Norte,JARDIM DO SERIDÓ,Rio Grande do Norte,JARDIM DO SERIDÓ,0.0,REPROVADO POR FALTAS,20091,ALGORITMOS E LÓGICA DE PROGRAMAÇÃO,90,2009,CANCELADO
1,bad4e0ec61e635f382e7e32c32d5ad11,2.0,"0,0",False,44.0,F,1991,Rio Grande do Norte,JARDIM DO SERIDÓ,Rio Grande do Norte,JARDIM DO SERIDÓ,0.0,REPROVADO POR FALTAS,20091,ALGORITMOS E LÓGICA DE PROGRAMAÇÃO,90,2009,CANCELADO
2,bad4e0ec61e635f382e7e32c32d5ad11,3.0,"0,0",False,44.0,F,1991,Rio Grande do Norte,JARDIM DO SERIDÓ,Rio Grande do Norte,JARDIM DO SERIDÓ,0.0,REPROVADO POR FALTAS,20091,ALGORITMOS E LÓGICA DE PROGRAMAÇÃO,90,2009,CANCELADO
3,bee926721a5b23a098fdfb850bf8d5b5,1.0,"0,0",False,52.0,M,1983,Rio Grande do Norte,CAICÓ,Rio Grande do Norte,JARDIM DE PIRANHAS,0.0,REPROVADO POR FALTAS,20091,ALGORITMOS E LÓGICA DE PROGRAMAÇÃO,90,2009,CANCELADO
4,bee926721a5b23a098fdfb850bf8d5b5,2.0,"0,0",False,52.0,M,1983,Rio Grande do Norte,CAICÓ,Rio Grande do Norte,JARDIM DE PIRANHAS,0.0,REPROVADO POR FALTAS,20091,ALGORITMOS E LÓGICA DE PROGRAMAÇÃO,90,2009,CANCELADO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36431,22f4aed4a073c5e9515a8669e9c102f3,2.0,"6,9",False,0.0,M,1984,Rio Grande do Norte,CAICÓ,Rio Grande do Norte,CAICÓ,5.2,APROVADO POR NOTA,20222,PROGRAMAÇÃO VISUAL,60,2019,ATIVO - FORMANDO
36432,22f4aed4a073c5e9515a8669e9c102f3,3.0,"5,0",False,0.0,M,1984,Rio Grande do Norte,CAICÓ,Rio Grande do Norte,CAICÓ,5.2,APROVADO POR NOTA,20222,PROGRAMAÇÃO VISUAL,60,2019,ATIVO - FORMANDO
36433,943463d2de8c6a60ec5f5d959ba7f1ac,1.0,"10,0",False,0.0,M,1999,Rio Grande do Norte,CAICÓ,Rio Grande do Norte,SÃO FERNANDO,10.0,APROVADO,20222,MATEMÁTICA FINANCEIRA,60,2018,CONCLUÍDO
36434,943463d2de8c6a60ec5f5d959ba7f1ac,2.0,"10,0",False,0.0,M,1999,Rio Grande do Norte,CAICÓ,Rio Grande do Norte,SÃO FERNANDO,10.0,APROVADO,20222,MATEMÁTICA FINANCEIRA,60,2018,CONCLUÍDO


Ampliando a vizualização das colunas no vscode.

In [238]:
pd.set_option('display.max_columns', 310)

# Tratamento de Dados

Listando as colunas do dataframe.

In [239]:
df_dados.columns

Index(['discente', 'unidade', 'nota', 'reposicao', 'numero_total_faltas',
       'sexo', 'ano_nascimento', 'estado_origem', 'cidade_origem', 'estado',
       'municipio', 'media_final', 'descricao', 'ano', 'nome', 'ch_total',
       'ano_ingresso', 'status'],
      dtype='object')

In [240]:

nomes = {
    'numero_total_faltas': 'frequencia'
}

df_dados.rename(columns=nomes, inplace=True)

Transformando a coluna *media_final* no tipo float.

In [241]:
df_dados['media_final']=df_dados['media_final'].apply(lambda x: str(x).replace(',' , '.'))
df_dados['media_final'] = df_dados['media_final'].astype('float64')

Informações sobre o tipo de cada coluna do dataframe.

In [242]:
df_dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36436 entries, 0 to 36435
Data columns (total 18 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   discente        36436 non-null  object 
 1   unidade         36436 non-null  float64
 2   nota            34010 non-null  object 
 3   reposicao       35727 non-null  object 
 4   frequencia      35463 non-null  float64
 5   sexo            36436 non-null  object 
 6   ano_nascimento  36436 non-null  int64  
 7   estado_origem   36197 non-null  object 
 8   cidade_origem   36153 non-null  object 
 9   estado          36152 non-null  object 
 10  municipio       36152 non-null  object 
 11  media_final     36436 non-null  float64
 12  descricao       36436 non-null  object 
 13  ano             36436 non-null  int64  
 14  nome            36436 non-null  object 
 15  ch_total        36436 non-null  int64  
 16  ano_ingresso    36436 non-null  int64  
 17  status          36436 non-null 

Listando os dados NaN.

In [243]:
df_dados.isnull().sum()

discente             0
unidade              0
nota              2426
reposicao          709
frequencia         973
sexo                 0
ano_nascimento       0
estado_origem      239
cidade_origem      283
estado             284
municipio          284
media_final          0
descricao            0
ano                  0
nome                 0
ch_total             0
ano_ingresso         0
status               0
dtype: int64

Preenchendo os dados NaN da coluna *unidade* com 1.

In [244]:
df_dados.loc[:, 'unidade'] = df_dados['unidade'].fillna(1)

Apagando da coluna *descricao* a condição *INDEFERIDO* e *EXCLUIDA*.

In [245]:
df_dados = df_dados[~df_dados['descricao'].isin(['INDEFERIDO', 'EXCLUIDA'])]

Verificar os valores faltosos na coluna *nota* com base na condição da coluna *descricao*.

In [246]:
condicao_cancelado = df_dados['descricao'] == 'CANCELADO'
df_dados.loc[condicao_cancelado, 'nota'] = -2

condicao_trancado = df_dados['descricao'] == 'TRANCADO'
df_dados.loc[condicao_trancado, 'nota'] = -4

condicao_desistencia = df_dados['descricao'] == 'DESISTENCIA'
df_dados.loc[condicao_desistencia, 'nota'] = -7

Verificar os valores faltosos na coluna *media_final* com base na condição da coluna *descricao*.

In [247]:
condicao_cancelado = df_dados['descricao'] == 'CANCELADO'
df_dados.loc[condicao_cancelado, 'media_final'] = -2

condicao_trancado = df_dados['descricao'] == 'TRANCADO'
df_dados.loc[condicao_trancado, 'media_final'] = -4

condicao_desistencia = df_dados['descricao'] == 'DESISTENCIA'
df_dados.loc[condicao_desistencia, 'media_final'] = -7

Verificar os valores faltosos na coluna *frequencia* com base na condição da coluna *descricao*.

In [248]:
condicao_cancelado = df_dados['descricao'] == 'CANCELADO'
df_dados.loc[condicao_cancelado, 'frequencia'] = -2

condicao_trancado = df_dados['descricao'] == 'TRANCADO'
df_dados.loc[condicao_trancado, 'frequencia'] = -4

condicao_desistencia = df_dados['descricao'] == 'DESISTENCIA'
df_dados.loc[condicao_desistencia, 'frequencia'] = -7

In [249]:
df_dados.isnull().sum()

discente            0
unidade             0
nota                0
reposicao         709
frequencia          0
sexo                0
ano_nascimento      0
estado_origem     239
cidade_origem     283
estado            284
municipio         284
media_final         0
descricao           0
ano                 0
nome                0
ch_total            0
ano_ingresso        0
status              0
dtype: int64

tratamento de dados faltosos

In [250]:
df_dados.loc[:, 'reposicao'] = df_dados['reposicao'].fillna(0)
df_dados.loc[:, 'estado_origem'] = df_dados['estado_origem'].fillna('Não informado')
df_dados.loc[:, 'cidade_origem'] = df_dados['cidade_origem'].fillna('Não informado')
df_dados.loc[:, 'estado'] = df_dados['estado'].fillna('Não informado')
df_dados.loc[:, 'municipio'] = df_dados['municipio'].fillna('Não informado')

In [251]:
df_dados.isnull().sum()

discente          0
unidade           0
nota              0
reposicao         0
frequencia        0
sexo              0
ano_nascimento    0
estado_origem     0
cidade_origem     0
estado            0
municipio         0
media_final       0
descricao         0
ano               0
nome              0
ch_total          0
ano_ingresso      0
status            0
dtype: int64

Contando os valores dos dados da coluna *descricao*.

In [252]:
df_dados.descricao.value_counts()

descricao
APROVADO                            17742
APROVADO POR NOTA                    6675
REPROVADO                            5163
REPROVADO POR MÉDIA E POR FALTAS     3045
CANCELADO                            2026
TRANCADO                             1349
DESISTENCIA                           208
REPROVADO POR FALTAS                  123
REPROVADO POR NOTA                     78
REPROVADO POR NOTA E FALTA             27
Name: count, dtype: int64

In [253]:
df_dados.to_csv('df_bsi_ORIGINAL_DEZEMBRO_limpo.csv', index=False, sep=',', quoting=csv.QUOTE_NONNUMERIC)

Criar uma função para mapear a classificação da coluna *descricao*.

In [254]:
def mapear_classificacao(descricao):
    mapeamento = {
        'APROVADO': 2,
        'APROVADO POR NOTA': 1,
        'REPROVADO': -1,
        'REPROVADO POR FALTAS': -1,
        'REPROVADO POR NOTA': -1,
        'REPROVADO POR MÉDIA E POR FALTAS': -2,
        'REPROVADO POR NOTA E FALTA': -2,
        'CANCELADO': -3,
        'EXCLUIDA': -3,
        'TRANCADO': -3,
        'DESISTENCIA': -4,
    }
    return mapeamento.get(descricao, None)

Aplicar a função de mapeamento e criar a coluna *classificacao*.

In [255]:
df_dados.loc[:, 'classificacao'] = df_dados['descricao'].map(mapear_classificacao)

Fazendo um recorte da nossa análise, vamos começar por disciplinas obrigatórias do Bacharelado em Sistemas da Informação (BSI):

In [256]:
lista_obrigatórias = [
                'ALGORITMOS E LÓGICA DE PROGRAMAÇÃO',
                'INTRODUÇÃO À INFORMÁTICA',
                'FUNDAMENTOS DE MATEMÁTICA',
                'LÓGICA',
                'TEORIA GERAL DA ADMINISTRAÇÃO',
                'PROGRAMAÇÃO',
                'CÁLCULO DIFERENCIAL E INTEGRAL',
                'TEORIA GERAL DOS SISTEMAS',
                'PROGRAMAÇÃO ORIENTADA A OBJETOS I',
                'ESTRUTURA DE DADOS',
                'ÁLGEBRA LINEAR',
                'ORGANIZAÇÃO, SISTEMAS E MÉTODOS',
                'FUNDAMENTOS DE SISTEMAS DE INFORMAÇÃO',
                'PROGRAMAÇÃO WEB',
                'ARQUITETURA DE COMPUTADORES',
                'PROBABILIDADE E ESTATÍSTICA',
                'BANCO DE DADOS',
                'ENGENHARIA DE SOFTWARE I',
                'PROGRAMAÇÃO ORIENTADA A OBJETOS II',
                'SISTEMAS OPERACIONAIS',
                'PROJETO E ADMINISTRAÇÃO DE BANCO DE DADOS',
                'ENGENHARIA DE SOFTWARE II',
                'REDES DE COMPUTADORES',
                'EMPREENDEDORISMO EM INFORMÁTICA',
                'GESTÃO DE PROJETO DE SOFTWARE',
                'PROGRAMAÇÃO VISUAL',
                'MATEMÁTICA FINANCEIRA',
                'SISTEMAS DE APOIO À DECISÃO',
                'ÉTICA',
                ]
condição_nome = f"nome in {lista_obrigatórias}"
df_dados_filtrado = df_dados.query(condição_nome)

# Extraindo dados

In [257]:
df_dados_filtrado.columns

Index(['discente', 'unidade', 'nota', 'reposicao', 'frequencia', 'sexo',
       'ano_nascimento', 'estado_origem', 'cidade_origem', 'estado',
       'municipio', 'media_final', 'descricao', 'ano', 'nome', 'ch_total',
       'ano_ingresso', 'status', 'classificacao'],
      dtype='object')

### Colunas com as **Disciplinas**

Filtrar os dados onde a coluna **unidade** foi preenchida com 1.

In [258]:
filtrando_unidade = df_dados_filtrado[df_dados_filtrado['unidade'] == 1]

In [259]:
filtrando_unidade.to_csv('df_bsi_ORIGINAL_DEZEMBRO_limpo2.csv', index=False, sep=',', quoting=csv.QUOTE_NONNUMERIC)

Calculando a quantidade de vezes que cada discente cursou cada *disciplina*.

In [260]:
quantidade_disciplinas = filtrando_unidade.groupby(['discente', 'nome']).size().reset_index(name='quantidade')


Pivotando as *disciplinas*.

In [261]:
tabela_final = quantidade_disciplinas.pivot(index='discente', columns='nome', values='quantidade').reset_index()

In [262]:
tabela_final.replace(0, -1, inplace=True)

Substituir NaN por 0 usando fillna.

In [263]:
tabela_final = tabela_final.fillna(0)

### Coluna com o **Ano de Nascimento**

Adicionando o *ano_nascimento* para cada discente.

In [264]:
ano_nascimento_discente = df_dados_filtrado.drop_duplicates(subset=['discente'])[['discente', 'ano_nascimento']]
tabela_final = tabela_final.merge(ano_nascimento_discente, on='discente', how='left')

### Coluna com o **Ano de Ingresso**

Adicionando o *ano_ingresso* para cada discente.

In [265]:
ano_ingresso_discente = df_dados_filtrado.drop_duplicates(subset=['discente'])[['discente', 'ano_ingresso']]
tabela_final = tabela_final.merge(ano_ingresso_discente, on='discente', how='left')

### Coluna com o **Sexo** do Discente

Definir a opção global para permitir o downcasting silencioso

In [266]:
pd.set_option('future.no_silent_downcasting', True)

Adicionar as colunas *mulher* e *homem* usando o método assign para evitar SettingWithCopyWarning.

In [267]:
"""df_dados_filtrado = df_dados_filtrado.assign(
    mulher=df_dados_filtrado['sexo'].map({0: 0, 1: 1}),
    homem=df_dados_filtrado['sexo'].map({0: 1, 1: 0})
)"""

"df_dados_filtrado = df_dados_filtrado.assign(\n    mulher=df_dados_filtrado['sexo'].map({0: 0, 1: 1}),\n    homem=df_dados_filtrado['sexo'].map({0: 1, 1: 0})\n)"

Binarização da coluna Sexo.

In [268]:
df_dados_filtrado['sexo'] = df_dados_filtrado['sexo'].replace({'M': 0, 'F': 1})

Criar DataFrames com discente e as novas colunas.

In [269]:
#sexo_discente = df_dados_filtrado.drop_duplicates(subset=['discente'])[['discente', 'mulher', 'homem']]
sexo_discente = df_dados_filtrado.drop_duplicates(subset=['discente'])[['discente', 'sexo']]

Mesclar com tabela_final.

In [270]:
tabela_final = tabela_final.merge(sexo_discente, on='discente', how='left')


### Coluna com o **Status** do Discente

Alterando o conteúdo da coluna *status*.

In [271]:
df_dados_filtrado['status'] = df_dados_filtrado['status'].replace({
    'CANCELADO': -1, 
    'ATIVO': 1, 'ATIVO - FORMANDO': 1, 'TRANCADO': 1,
    'FORMADO': 1,'CONCLUÍDO': 1})

In [272]:
df_dados_filtrado.drop(df_dados_filtrado[df_dados_filtrado['status'] == 0].index, inplace=True)

In [273]:
df_dados_filtrado

,discente,unidade,nota,reposicao,frequencia,sexo,ano_nascimento,estado_origem,cidade_origem,estado,municipio,media_final,descricao,ano,nome,ch_total,ano_ingresso,status,classificacao
0,bad4e0ec61e635f382e7e32c32d5ad11,1.0,"0,0",False,44.0,1,1991,Rio Grande do Norte,JARDIM DO SERIDÓ,Rio Grande do Norte,JARDIM DO SERIDÓ,0.0,REPROVADO POR FALTAS,20091,ALGORITMOS E LÓGICA DE PROGRAMAÇÃO,90,2009,-1,-1
1,bad4e0ec61e635f382e7e32c32d5ad11,2.0,"0,0",False,44.0,1,1991,Rio Grande do Norte,JARDIM DO SERIDÓ,Rio Grande do Norte,JARDIM DO SERIDÓ,0.0,REPROVADO POR FALTAS,20091,ALGORITMOS E LÓGICA DE PROGRAMAÇÃO,90,2009,-1,-1
2,bad4e0ec61e635f382e7e32c32d5ad11,3.0,"0,0",False,44.0,1,1991,Rio Grande do Norte,JARDIM DO SERIDÓ,Rio Grande do Norte,JARDIM DO SERIDÓ,0.0,REPROVADO POR FALTAS,20091,ALGORITMOS E LÓGICA DE PROGRAMAÇÃO,90,2009,-1,-1
3,bee926721a5b23a098fdfb850bf8d5b5,1.0,"0,0",False,52.0,0,1983,Rio Grande do Norte,CAICÓ,Rio Grande do Norte,JARDIM DE PIRANHAS,0.0,REPROVADO POR FALTAS,20091,ALGORITMOS E LÓGICA DE PROGRAMAÇÃO,90,2009,-1,-1
4,bee926721a5b23a098fdfb850bf8d5b5,2.0,"0,0",False,52.0,0,1983,Rio Grande do Norte,CAICÓ,Rio Grande do Norte,JARDIM DE PIRANHAS,0.0,REPROVADO POR FALTAS,20091,ALGORITMOS E LÓGICA DE PROGRAMAÇÃO,90,2009,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36431,22f4aed4a073c5e9515a8669e9c102f3,2.0,"6,9",False,0.0,0,1984,Rio Grande do Norte,CAICÓ,Rio Grande do Norte,CAICÓ,5.2,APROVADO POR NOTA,20222,PROGRAMAÇÃO VISUAL,60,2019,1,1
36432,22f4aed4a073c5e9515a8669e9c102f3,3.0,"5,0",False,0.0,0,1984,Rio Grande do Norte,CAICÓ,Rio Grande do Norte,CAICÓ,5.2,APROVADO POR NOTA,20222,PROGRAMAÇÃO VISUAL,60,2019,1,1
36433,943463d2de8c6a60ec5f5d959ba7f1ac,1.0,"10,0",False,0.0,0,1999,Rio Grande do Norte,CAICÓ,Rio Grande do Norte,SÃO FERNANDO,10.0,APROVADO,20222,MATEMÁTICA FINANCEIRA,60,2018,1,2
36434,943463d2de8c6a60ec5f5d959ba7f1ac,2.0,"10,0",False,0.0,0,1999,Rio Grande do Norte,CAICÓ,Rio Grande do Norte,SÃO FERNANDO,10.0,APROVADO,20222,MATEMÁTICA FINANCEIRA,60,2018,1,2


In [274]:
status_data = df_dados_filtrado['status'].value_counts()
status_data

status
-1    18358
1     18078
Name: count, dtype: int64

In [275]:
df_dados_filtrado = df_dados_filtrado[df_dados_filtrado['status'] != 0]

In [276]:
df_dados_filtrado

,discente,unidade,nota,reposicao,frequencia,sexo,ano_nascimento,estado_origem,cidade_origem,estado,municipio,media_final,descricao,ano,nome,ch_total,ano_ingresso,status,classificacao
0,bad4e0ec61e635f382e7e32c32d5ad11,1.0,"0,0",False,44.0,1,1991,Rio Grande do Norte,JARDIM DO SERIDÓ,Rio Grande do Norte,JARDIM DO SERIDÓ,0.0,REPROVADO POR FALTAS,20091,ALGORITMOS E LÓGICA DE PROGRAMAÇÃO,90,2009,-1,-1
1,bad4e0ec61e635f382e7e32c32d5ad11,2.0,"0,0",False,44.0,1,1991,Rio Grande do Norte,JARDIM DO SERIDÓ,Rio Grande do Norte,JARDIM DO SERIDÓ,0.0,REPROVADO POR FALTAS,20091,ALGORITMOS E LÓGICA DE PROGRAMAÇÃO,90,2009,-1,-1
2,bad4e0ec61e635f382e7e32c32d5ad11,3.0,"0,0",False,44.0,1,1991,Rio Grande do Norte,JARDIM DO SERIDÓ,Rio Grande do Norte,JARDIM DO SERIDÓ,0.0,REPROVADO POR FALTAS,20091,ALGORITMOS E LÓGICA DE PROGRAMAÇÃO,90,2009,-1,-1
3,bee926721a5b23a098fdfb850bf8d5b5,1.0,"0,0",False,52.0,0,1983,Rio Grande do Norte,CAICÓ,Rio Grande do Norte,JARDIM DE PIRANHAS,0.0,REPROVADO POR FALTAS,20091,ALGORITMOS E LÓGICA DE PROGRAMAÇÃO,90,2009,-1,-1
4,bee926721a5b23a098fdfb850bf8d5b5,2.0,"0,0",False,52.0,0,1983,Rio Grande do Norte,CAICÓ,Rio Grande do Norte,JARDIM DE PIRANHAS,0.0,REPROVADO POR FALTAS,20091,ALGORITMOS E LÓGICA DE PROGRAMAÇÃO,90,2009,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36431,22f4aed4a073c5e9515a8669e9c102f3,2.0,"6,9",False,0.0,0,1984,Rio Grande do Norte,CAICÓ,Rio Grande do Norte,CAICÓ,5.2,APROVADO POR NOTA,20222,PROGRAMAÇÃO VISUAL,60,2019,1,1
36432,22f4aed4a073c5e9515a8669e9c102f3,3.0,"5,0",False,0.0,0,1984,Rio Grande do Norte,CAICÓ,Rio Grande do Norte,CAICÓ,5.2,APROVADO POR NOTA,20222,PROGRAMAÇÃO VISUAL,60,2019,1,1
36433,943463d2de8c6a60ec5f5d959ba7f1ac,1.0,"10,0",False,0.0,0,1999,Rio Grande do Norte,CAICÓ,Rio Grande do Norte,SÃO FERNANDO,10.0,APROVADO,20222,MATEMÁTICA FINANCEIRA,60,2018,1,2
36434,943463d2de8c6a60ec5f5d959ba7f1ac,2.0,"10,0",False,0.0,0,1999,Rio Grande do Norte,CAICÓ,Rio Grande do Norte,SÃO FERNANDO,10.0,APROVADO,20222,MATEMÁTICA FINANCEIRA,60,2018,1,2


Adicionando o *status* para cada discente.


In [277]:
status_discente = df_dados_filtrado.drop_duplicates(subset=['discente'])[['discente', 'status']]
tabela_final = tabela_final.merge(status_discente, on='discente', how='left')

### Coluna com o **classificacao** da Descrição

Adicionando o *classificacao* para cada discente.

In [278]:
classificacao_descricao = df_dados_filtrado.drop_duplicates(subset=['discente'])[['discente', 'classificacao']]
tabela_final = tabela_final.merge(classificacao_descricao, on='discente', how='left')

### Coluna com a **Carga Horária Cumprida**

In [279]:
filtrando_unidade = df_dados_filtrado[df_dados_filtrado['unidade'] == 1]

In [280]:
df_dados_filtrado.columns

Index(['discente', 'unidade', 'nota', 'reposicao', 'frequencia', 'sexo',
       'ano_nascimento', 'estado_origem', 'cidade_origem', 'estado',
       'municipio', 'media_final', 'descricao', 'ano', 'nome', 'ch_total',
       'ano_ingresso', 'status', 'classificacao'],
      dtype='object')

Agrupando por discente e somando a *carga horária*.

In [281]:
df_carga_horaria_cumprida = filtrando_unidade.groupby('discente')['ch_total'].sum().reset_index()

Adicionando a *carga horária cumprida* de cada discente e renomeando a coluna.

In [282]:
df_carga_horaria_cumprida.rename(columns={'ch_total': 'ch_cumprida'}, inplace=True)
tabela_final = tabela_final.merge(df_carga_horaria_cumprida, on='discente', how='left')

Dividindo a *carga horária cumprida* do discente pela carga horária das disciplinas obrigatórias.

In [283]:
tabela_final['ch_cumprida_dividida'] = tabela_final['ch_cumprida'] / 1830

### Coluna com a **Carga Horária Cumprida** com **Reprovação**

Filtrando dados pela unidade.

In [284]:
filtrando_unidade = df_dados_filtrado[df_dados_filtrado['unidade'] == 1]

Filtrar as disciplinas *reprovadas*.

In [285]:
reprovadas_status = [
    'REPROVADO',
    'REPROVADO POR NOTA',
    'REPROVADO POR FALTAS',
    'REPROVADO POR NOTA E FALTA',
    'REPROVADO POR MÉDIA E POR FALTAS'
]

In [286]:
df_reprovadas = filtrando_unidade[filtrando_unidade['descricao'].isin(reprovadas_status)]

Agrupar por discente e somar a *carga horária das disciplinas reprovadas*.

In [287]:
ch_reprovacao = df_reprovadas.groupby('discente')['ch_total'].sum().reset_index()

Renomear a coluna resultante.

In [288]:
ch_reprovacao.columns = ['discente', 'ch_reprovacao']

Adicionando a *carga horária de reprovação* de cada discente.

In [289]:
tabela_final = tabela_final.merge(ch_reprovacao, on='discente', how='left')
df_dados_filtrado = df_dados_filtrado.merge(ch_reprovacao, on='discente', how='left')

### Coluna com **taxa de reprovação**

In [290]:
tabela_final['taxa_reprovacao'] = tabela_final['ch_reprovacao'] / tabela_final['ch_cumprida'] 

### Coluna com o **Semestre**

Função para calcular o *semestre* com base no *ano de ingresso* e o *ano cursado.*

In [291]:
def calcular_semestre(ano_ingresso, ano_cursado):
    try:

        # Convertendo os anos para strings e ajustando para o formato correto
        ano_ingresso_str = str(ano_ingresso) + '1' if len(str(ano_ingresso)) == 4 else str(ano_ingresso)
        ano_cursado_str = str(ano_cursado) + '1' if len(str(ano_cursado)) == 4 else str(ano_cursado)
        
        ano_ingresso_num = int(ano_ingresso_str[:4])
        semestre_ingresso = int(ano_ingresso_str[4])
        
        ano_cursado_num = int(ano_cursado_str[:4])
        semestre_cursado = int(ano_cursado_str[4])
        
        anos_passados = (ano_cursado_num - ano_ingresso_num) * 2
        semestres_passados = anos_passados + (semestre_cursado - semestre_ingresso)
        
        return semestres_passados + 1  # +1 porque o semestre de ingresso é contado como o primeiro
    except (IndexError, ValueError):
        return pd.NA  # Retorna NA se houver um problema com o formato dos anos

Substituir valores específicos na coluna *ano*

In [292]:
df_dados_filtrado.loc[df_dados_filtrado['ano'] == 20205, 'ano'] = 20203
df_dados_filtrado.loc[df_dados_filtrado['ano'] == 20206, 'ano'] = 20204

Corrigir os valores das colunas *ano_ingresso* e *ano*.

In [293]:
df_dados_filtrado['ano_ingresso'] = df_dados_filtrado['ano_ingresso'].astype(str)
df_dados_filtrado['ano'] = df_dados_filtrado['ano'].astype(str)

Aplicar a função para criar a coluna *semestre*.

In [294]:
df_dados_filtrado['semestre'] = df_dados_filtrado.apply(
    lambda x: calcular_semestre(x['ano_ingresso'], x['ano']), axis=1
)

In [295]:
# Agrupar por discente e disciplina e coletar os semestres
semestres_por_disciplina = df_dados_filtrado.groupby(['discente', 'nome'])['semestre'].apply(lambda x: ', '.join(map(str, x.unique()))).reset_index()

# Renomear a coluna para algo mais descritivo
semestres_por_disciplina.rename(columns={'semestre': 'semestres_cursados'}, inplace=True)

In [296]:
# Supondo que tabela_final já existe e tem as colunas 'discente' e 'disciplina'
df_dados_filtrado = df_dados_filtrado.merge(semestres_por_disciplina, on=['discente' , 'nome'], how='left') 

In [297]:
semestres = df_dados_filtrado[['discente', 'semestre']].drop_duplicates("discente")
tabela_final = tabela_final.merge(semestres, on='discente', how='left')


### Coluna com o Total de **Semestre Cursado**

Identificar *semestres* únicos cursados por cada discente.

In [298]:
semestres_unicos_por_discente = df_dados_filtrado.groupby('discente')['ano'].nunique().reset_index()
semestres_unicos_por_discente.rename(columns={'ano' : 'semestre_total'}, inplace=True)

Adicionando a quantidade de *semestres* cursados por cada discente.

In [299]:
tabela_final = tabela_final.merge(semestres_unicos_por_discente, on='discente', how='left')

Dividindo a quantidade de *semestres cursados* pela quantidade de *semestres estimados* no curso.

In [300]:
tabela_final['semestre_dividido'] = tabela_final['semestre_total'] / 8

### Coluna com o **Último Período** Matrículado

Encontrar o *último período* que cada aluno estudou no curso.

In [301]:
ultimo_periodo = df_dados_filtrado.groupby('discente')['ano'].max().reset_index()
ultimo_periodo.rename(columns={'ano': 'ultimo_periodo'}, inplace=True)

Adicionando as informações do *último período* de cada discente.

In [302]:
tabela_final = tabela_final.merge(ultimo_periodo, on='discente', how='left')

### Coluna com a quantidade de vezes que o discente se **Matriculou**

Soma dos valores das colunas para contabilizar a *matrícula* por discente.

In [303]:
matricula =      tabela_final['ALGORITMOS E LÓGICA DE PROGRAMAÇÃO'] + \
                 tabela_final['INTRODUÇÃO À INFORMÁTICA'] + \
                 tabela_final['FUNDAMENTOS DE MATEMÁTICA'] + \
                 tabela_final['LÓGICA'] + \
                 tabela_final['TEORIA GERAL DA ADMINISTRAÇÃO'] + \
                 tabela_final['PROGRAMAÇÃO'] + \
                 tabela_final['CÁLCULO DIFERENCIAL E INTEGRAL'] + \
                 tabela_final['TEORIA GERAL DOS SISTEMAS'] + \
                 tabela_final['PROGRAMAÇÃO ORIENTADA A OBJETOS I'] + \
                 tabela_final['ESTRUTURA DE DADOS'] + \
                 tabela_final['ÁLGEBRA LINEAR'] + \
                 tabela_final['ORGANIZAÇÃO, SISTEMAS E MÉTODOS'] + \
                 tabela_final['FUNDAMENTOS DE SISTEMAS DE INFORMAÇÃO'] + \
                 tabela_final['PROGRAMAÇÃO WEB'] + \
                 tabela_final['ARQUITETURA DE COMPUTADORES'] + \
                 tabela_final['PROBABILIDADE E ESTATÍSTICA'] + \
                 tabela_final['BANCO DE DADOS'] + \
                 tabela_final['ENGENHARIA DE SOFTWARE I'] + \
                 tabela_final['PROGRAMAÇÃO ORIENTADA A OBJETOS II'] + \
                 tabela_final['SISTEMAS OPERACIONAIS'] + \
                 tabela_final['PROJETO E ADMINISTRAÇÃO DE BANCO DE DADOS'] + \
                 tabela_final['ENGENHARIA DE SOFTWARE II'] + \
                 tabela_final['REDES DE COMPUTADORES'] + \
                 tabela_final['EMPREENDEDORISMO EM INFORMÁTICA'] + \
                 tabela_final['GESTÃO DE PROJETO DE SOFTWARE'] + \
                 tabela_final['PROGRAMAÇÃO VISUAL'] + \
                 tabela_final['MATEMÁTICA FINANCEIRA'] + \
                 tabela_final['SISTEMAS DE APOIO À DECISÃO'] + \
                 tabela_final['ÉTICA']

Adicionando a coluna *matrícula* na tabela.

In [304]:
tabela_final['matricula'] = matricula

In [305]:
dados_da_coluna = tabela_final['matricula']

### Coluna com o **Tempo Relativo** que o discente passou no curso

Converter a coluna *ch_cumprida_dividida* para tipo numérico.

In [306]:
tabela_final['semestre_dividido'] = pd.to_numeric(tabela_final['semestre_dividido'], errors='coerce')

Converter a coluna *status* para tipo numérico.

In [307]:
tabela_final['status'] = pd.to_numeric(tabela_final['status'], errors='coerce')

Multiplicar os valores da coluna *ch_cumprida_dividida* pela coluna *status*.

In [308]:
tabela_final['tempo_relativo'] = tabela_final['semestre_dividido'] * tabela_final['status']

In [309]:
tabela_final = tabela_final[tabela_final['tempo_relativo'] != 0]

### Coluna com a quantidade de vezes que o discente foi **Aprovado**

Filtrando dados pela unidade.

In [310]:
filtrando_unidade = df_dados_filtrado[df_dados_filtrado['unidade'] == 1]

Listar todas as disciplinas.

In [311]:
disciplinas = df_dados_filtrado['nome'].unique()

Criar uma tabela de *aprovações*.

In [312]:
aprovacoes = filtrando_unidade[filtrando_unidade['descricao'].isin([
    'APROVADO',
    'APROVADO POR NOTA',
])]

Agrupar por discente e nome da disciplina para contar as *aprovações*.

In [313]:
aprovacoes_count = aprovacoes.groupby(['discente', 'nome']).size().unstack(fill_value=0)

Renomear as colunas para incluir *_APROVADO*.

In [314]:
aprovacoes_count.columns = [f"{disciplina}_APROVADO" for disciplina in aprovacoes_count.columns]

Mesclar as *aprovações* de volta ao DataFrame original.

In [315]:
# Criar `df_final` removendo duplicatas em 'discente' e juntando com `aprovacoes_count`
df_final = df_dados_filtrado.drop_duplicates('discente').set_index('discente').join(aprovacoes_count, on='discente').fillna(-1).reset_index()

# Selecionar as colunas de df_final que não estão presentes em tabela_final
colunas_para_juntar = [col for col in df_final.columns if col not in tabela_final.columns or col == 'discente']

# Selecionar apenas as colunas necessárias
df_final_rename = df_final[colunas_para_juntar]

# Realizar a junção com tabela_final
tabela_final = tabela_final.merge(df_final_rename, on='discente', how='left')

### Coluna com a quantidade de vezes que o discente foi **Reprovado**

Listar todas as disciplinas.

In [316]:
disciplinas = df_dados_filtrado['nome'].unique()

Criar uma tabela de *reprovações* com o filtro da 1 unidade.

In [317]:
reprovacoes = filtrando_unidade[filtrando_unidade['descricao'].isin([
    'REPROVADO',
    'REPROVADO POR NOTA',
    'REPROVADO POR FALTAS',
    'REPROVADO POR NOTA E FALTA',
    'REPROVADO POR MÉDIA E POR FALTAS'
])]

Agrupar por discente e nome da disciplina para contar as *reprovações*.

In [318]:
reprovacoes_count = reprovacoes.groupby(['discente', 'nome']).size().unstack(fill_value=0)

Renomear as colunas para incluir *_REPROVADO*.

In [319]:
reprovacoes_count.columns = [f"{disciplina}_REPROVADO" for disciplina in reprovacoes_count.columns]

Mesclar as *reprovações* de volta ao DataFrame original.

In [320]:
# Criar `df_final` removendo duplicatas em 'discente' e juntando com `reprovacoes_count`
df_final = df_dados_filtrado.drop_duplicates('discente').set_index('discente').join(reprovacoes_count, on='discente').fillna(-1).reset_index()

# Selecionar as colunas de df_final que não estão presentes em tabela_final
colunas_para_juntar = [col for col in df_final.columns if col not in tabela_final.columns or col == 'discente']

# Selecionar apenas as colunas necessárias
df_final_rename = df_final[colunas_para_juntar]

# Realizar a junção com tabela_final
tabela_final = tabela_final.merge(df_final_rename, on='discente', how='left')

In [321]:
# Dicionário de importância baseado na proximidade com o RN
estado_origem = {
    'Rio Grande do Norte': 1,
    'Paraíba': 2,
    'Ceará': 3,
    'Pernambuco': 4,
    'Alagoas': 5,
    'Sergipe': 6,
    'Bahia': 7,
    'Piauí': 8,
    'Maranhão': 9,
    'Tocantins': 10,
    'Goiás': 11,
    'Distrito Federal': 12,
    'Minas Gerais': 13,
    'Espírito Santo': 14,
    'Rio de Janeiro': 15,
    'São Paulo': 16,
    'Paraná': 17,
    'Santa Catarina': 18,
    'Rio Grande do Sul': 19,
    'Mato Grosso': 20,
    'Mato Grosso do Sul': 21,
    'Rondônia': 22,
    'Acre': 23,
    'Amazonas': 24,
    'Roraima': 25,
    'Pará': 26,
    'Amapá': 27,
    'Não informado': 28  # Não informado ou estados desconhecidos
}

# Priorizar estado_origem e usar estado como fallback, utilizando .loc
tabela_final.loc[:, 'estado_origem'] = (
    tabela_final['estado_origem'].map(estado_origem)
    .fillna(28)  # Valor padrão para casos não mapeados
)

In [322]:
# Dicionário de importância baseado na proximidade com o RN
estado = {
    'Rio Grande do Norte': 1,
    'Paraíba': 2,
    'Ceará': 3,
    'Pernambuco': 4,
    'Alagoas': 5,
    'Sergipe': 6,
    'Bahia': 7,
    'Piauí': 8,
    'Maranhão': 9,
    'Tocantins': 10,
    'Goiás': 11,
    'Distrito Federal': 12,
    'Minas Gerais': 13,
    'Espírito Santo': 14,
    'Rio de Janeiro': 15,
    'São Paulo': 16,
    'Paraná': 17,
    'Santa Catarina': 18,
    'Rio Grande do Sul': 19,
    'Mato Grosso': 20,
    'Mato Grosso do Sul': 21,
    'Rondônia': 22,
    'Acre': 23,
    'Amazonas': 24,
    'Roraima': 25,
    'Pará': 26,
    'Amapá': 27,
    'Não informado': 28  # Não informado ou estados desconhecidos
}

# Priorizar estado e usar estado como fallback, utilizando .loc
tabela_final.loc[:, 'estado'] = (
    tabela_final['estado'].map(estado)
    .fillna(28)) # Valor padrão para casos não mapeados

In [323]:
cidade_origem= {
    "ACARI": 2,
    "APODI": 3,
    "AQUIDAUANA": 4,
    "BARUERI": 5,
    "BARCELONA": 6,
    "BREJO DO CRUZ": 7,
    "BRASÍLIA": 8,
    "CAMPINA GRANDE": 9,
    "CARNAÚBA DOS DANTAS": 10,
    "CAICÓ": 1,
    "CAIEIRAS": 11,
    "CAMPO FORMOSO": 12,
    "CAORÃ": 13,
    "CATOLÉ DO ROCHA": 14,
    "CEARÁ - MIRIM": 15,
    "CRUZETA": 16,
    "CURRAIS NOVOS": 17,
    "EQUADOR": 18,
    "FLORÂNIA": 19,
    "FORTALEZA": 20,
    "GUARULHOS": 21,

    "JACINTO": 23,
    "JARDIM DE PIRANHAS": 24,
    "JADIM DE PIRANHAS": 24,
    "JARDIM DO SERIDÓ": 25,
    "JANDUÍS": 26,
    "JUCURUTU": 27,
    "JUPIÁ": 28,
    "MACAU": 29,
    "MESSEIAS TARGINO": 30,
    "MONTES CLAROS": 31,
    "MOSSORÓ": 32,
    "NATAL": 33,

    "OURO BRANCO": 35,
    "PARELHAS": 36,
    "PATOS": 37,
    "PAULISTA": 38,
    "PICUÍ": 39,
    "PORTO VELHO": 40,
    "PARNAMIRIM": 41,
    "REGISTRO": 42,
    "RIO DE JANEIRO": 43,
    "SANTA CRUZ": 44,
    "SANTA LUZIA": 45,
    "SÃO BENTO": 46,
    "SÃO FERNANDO": 47,
    "SÃO GABRIEL DA CACHOEIRA": 48,
    "SÃO JOSÉ DO EGITO": 49,
    "SÃO JOSÉ DOS CAMPOS": 50,
    "SÃO JOÃO DO SABUGI": 51,
    "SÃO MAMEDE": 52,
    "SÃO MIGUEL": 53,
    "SÃO PAULO": 54,
    "SÃO RAFAEL": 55,
    "SERGIPE": 56,
    "SERRA NEGRA DO NORTE": 57,
    "TIMBAÚBA DOS BATISTAS": 58,
    "TUCURUÍ": 59,
    "VITÓRIA DE SANTO ANTÃO": 60,
    "VÁRZEA": 61,
    "NÃO INFORMADO":62
}

    
# Priorizar cidade_origem e usar estado como fallback, utilizando .loc
tabela_final.loc[:, 'cidade_origem'] = (
    tabela_final['cidade_origem'].map(cidade_origem)
    .fillna(62)  # Valor padrão para casos não mapeados
)


In [324]:
    municipio = {
    "ACARI": 2,
    "APODI": 3,
    "AQUIDAUANA": 4,
    "BARUERI": 5,
    "BARCELONA": 6,
    "BREJO DO CRUZ": 7,
    "BRASÍLIA": 8,
    "CAMPINA GRANDE": 9,
    "CARNAÚBA DOS DANTAS": 10,
    "CAICÓ": 1,
    "CAIEIRAS": 11,
    "CAMPO FORMOSO": 12,
    "CAORÃ": 13,
    "CATOLÉ DO ROCHA": 14,
    "CEARÁ - MIRIM": 15,
    "CRUZETA": 16,
    "CURRAIS NOVOS": 17,
    "EQUADOR": 18,
    "FLORÂNIA": 19,
    "FORTALEZA": 20,
    "GUARULHOS": 21,

    "JACINTO": 23,
    "JARDIM DE PIRANHAS": 24,
    "JADIM DE PIRANHAS": 24,
    "JARDIM DO SERIDÓ": 25,
    "JANDUÍS": 26,
    "JUCURUTU": 27,
    "JUPIÁ": 28,
    "MACAU": 29,
    "MESSEIAS TARGINO": 30,
    "MONTES CLAROS": 31,
    "MOSSORÓ": 32,
    "NATAL": 33,

    "OURO BRANCO": 35,
    "PARELHAS": 36,
    "PATOS": 37,
    "PAULISTA": 38,
    "PICUÍ": 39,
    "PORTO VELHO": 40,
    "PARNAMIRIM": 41,
    "REGISTRO": 42,
    "RIO DE JANEIRO": 43,
    "SANTA CRUZ": 44,
    "SANTA LUZIA": 45,
    "SÃO BENTO": 46,
    "SÃO FERNANDO": 47,
    "SÃO GABRIEL DA CACHOEIRA": 48,
    "SÃO JOSÉ DO EGITO": 49,
    "SÃO JOSÉ DOS CAMPOS": 50,
    "SÃO JOÃO DO SABUGI": 51,
    "SÃO MAMEDE": 52,
    "SÃO MIGUEL": 53,
    "SÃO PAULO": 54,
    "SÃO RAFAEL": 55,
    "SERGIPE": 56,
    "SERRA NEGRA DO NORTE": 57,
    "TIMBAÚBA DOS BATISTAS": 58,
    "TUCURUÍ": 59,
    "VITÓRIA DE SANTO ANTÃO": 60,
    "VÁRZEA": 61,
    "NÃO INFORMADO":62,
    "SANTA TERESINHA": 63,
    "VÁRZEA": 64,
    "ÁGUAS VERMELHAS": 65,
    "MATURÉIA": 66,
    "CAUCAIA": 67,
    "BELÉM DO BREJO DO CRUZ": 68,
    "CAMPO REDONDO": 69,
    "ALTO DO RODRIGUES": 70,
    "FREI MARTINHO": 71,
    "TEIXEIRA": 72,
    "JOÃO PESSOA": 73
}

    
# Priorizar municipio e usar estado como fallback, utilizando .loc
tabela_final.loc[:, 'municipio'] = (
    tabela_final['municipio'].map(municipio)
    .fillna(62)  # Valor padrão para casos não mapeados
)

    

## Pivoteamento

Pivoteando *Faltas*.

In [325]:
df_pivot_f = df_dados_filtrado.pivot_table(index='discente', columns='nome', values='frequencia', aggfunc='first')
df_pivot_f = df_pivot_f.add_suffix('_FALTAS')


Pivoteando *Notas*

In [326]:
'''df_pivot_nota = df_dados_filtrado.pivot_table(index='discente', columns= ['nome','unidade'], values='nota', aggfunc=lambda x: x.iloc[-1])
df_pivot_nota.columns = df_pivot_nota.columns.set_levels(
    [level + '_NOTA_' for level in df_pivot_nota.columns.levels[0]],  # Aplicando o sufixo aos nomes do nível 'nome'
    level=0  # Indicando que estamos modificando o primeiro nível (nome)
)'''


df_pivot_nota = df_dados_filtrado.pivot_table(index='discente', columns= ['nome','unidade'], values='nota', aggfunc='first')
df_pivot_nota.columns = df_pivot_nota.columns.set_levels(
    [level + '_NOTA_' for level in df_pivot_nota.columns.levels[0]],  # Aplicando o sufixo aos nomes do nível 'nome'
    level=0  # Indicando que estamos modificando o primeiro nível (nome)
)

Achate as colunas juntando diferentes níveis.

In [327]:
df_pivot_nota.columns = [''.join(map(str, col)).strip() for col in df_pivot_nota.columns.values]

Criação da tabela pivô com o último valor de *media_final*.

In [328]:
"""df_pivot_mf = df_dados_filtrado.pivot_table(index='discente', columns='nome', values='media_final', aggfunc=lambda x: x.iloc[-1])
df_pivot_mf = df_pivot_mf.add_suffix('_MEDIA_FINAL_1.0')"""

df_pivot_mf = df_dados_filtrado.pivot_table(index='discente', columns='nome', values='media_final', aggfunc='first')
df_pivot_mf = df_pivot_mf.add_suffix('_MEDIA_FINAL_1.0')

Criação da tabela pivô com a média da *media_final*.

In [329]:
df_pivot_mf_2 = df_dados_filtrado.pivot_table(index='discente', columns='nome', values='media_final', aggfunc='mean')
df_pivot_mf_2 = df_pivot_mf_2.add_suffix('_MEDIA_FINAL_2.0')

Criação da tabela pivô com o último valor de semestre

In [330]:
"""
df_pivot_sem = filtrando_unidade.pivot_table(index='discente', 
                                                   columns='nome', 
                                                   values='semestre', 
                                                   aggfunc=lambda x: x.iloc[-1] if len(x) > 0 else np.nan)
df_pivot_sem = df_pivot_sem.add_suffix('_SEMESTRE')"""


df_pivot_sem = filtrando_unidade.pivot_table(index='discente', 
                                                   columns='nome', 
                                                   values='semestre', 
                                                   aggfunc='first')
df_pivot_sem = df_pivot_sem.add_suffix('_SEMESTRE')

In [331]:
df_pivot_ch = df_dados_filtrado.pivot_table(index='discente', columns='nome', values='ch_total', aggfunc='first')
df_pivot_ch = df_pivot_ch.add_suffix('_CH_TOTAL')

In [332]:
df_pivot_rep = df_dados_filtrado.pivot_table(index='discente', columns='nome', values='reposicao', aggfunc='first')
df_pivot_rep = df_pivot_rep.add_suffix('_REPOSICAO')
df_pivot_rep = df_pivot_rep.fillna(-1)

# Transformar True em 1 e False em 0 no DataFrame
df_pivot_rep = df_pivot_rep.replace({True: 1, False: 0})

Juntando as tabelas anteriores com merge.

In [333]:
df_merged_1 = pd.merge(df_pivot_f, df_pivot_nota, on='discente')
df_merged_2 = pd.merge(df_merged_1, df_pivot_mf, on='discente')
df_merged_3 = pd.merge(df_merged_2, df_pivot_mf_2, on='discente')
df_merged_4 = pd.merge(df_merged_3, df_pivot_sem, on='discente')
df_merged_5 = pd.merge(df_merged_4, df_pivot_ch, on='discente')
df_merged_6 = pd.merge(df_merged_5, df_pivot_rep, on='discente')

df_merged_7 = pd.merge(df_merged_6, tabela_final, on='discente')


Substituir todos os valores 0 por -1.

In [334]:
df_merged_4.replace(0, -1, inplace=True)

Substituindo NaN por 0 nas colunas.

In [335]:
df_merged_7 = df_merged_7.fillna(0)


Arredondando todas as colunas para 1 casa decimal.

In [336]:
df_merged_7 = df_merged_7.round(2)

In [337]:
# Renomear colunas para nomes mais amigáveis
nomes = {
    'ch_cumprida_dividida': 'permanencia',
    'semestre_dividido': 'permanencia_por_semestre',
    'matricula': 'quant_matricula'
}
df_merged_7.rename(columns=nomes, inplace=True)

# Selecionar as colunas relevantes
description = [
    'permanencia', 
    'taxa_reprovacao', 
    'permanencia_por_semestre', 
    'quant_matricula', 
    'status'
]
df_dados = df_merged_7[description]
df_dados

,permanencia,taxa_reprovacao,permanencia_por_semestre,quant_matricula,status
0,0.18,0.00,0.12,5.0,-1
1,1.18,0.44,1.12,33.0,-1
2,0.48,0.31,0.38,13.0,-1
3,1.26,0.23,1.25,36.0,1
4,0.13,0.00,0.38,4.0,-1
...,...,...,...,...,...
653,0.30,0.50,0.25,8.0,1
654,1.33,0.20,1.38,38.0,1
655,0.74,0.00,0.88,21.0,1
656,1.46,0.24,1.50,41.0,1


In [338]:
df_dados.to_csv('df_CONJUNTO_B.csv', index=False, sep=',', quoting=csv.QUOTE_NONNUMERIC)

In [339]:
print(df_merged_7.columns)

Index(['discente', 'ALGORITMOS E LÓGICA DE PROGRAMAÇÃO_FALTAS',
       'ARQUITETURA DE COMPUTADORES_FALTAS', 'BANCO DE DADOS_FALTAS',
       'CÁLCULO DIFERENCIAL E INTEGRAL_FALTAS',
       'EMPREENDEDORISMO EM INFORMÁTICA_FALTAS',
       'ENGENHARIA DE SOFTWARE I_FALTAS', 'ENGENHARIA DE SOFTWARE II_FALTAS',
       'ESTRUTURA DE DADOS_FALTAS', 'FUNDAMENTOS DE MATEMÁTICA_FALTAS',
       ...
       'PROGRAMAÇÃO VISUAL_REPROVADO', 'PROGRAMAÇÃO WEB_REPROVADO',
       'PROJETO E ADMINISTRAÇÃO DE BANCO DE DADOS_REPROVADO',
       'REDES DE COMPUTADORES_REPROVADO',
       'SISTEMAS DE APOIO À DECISÃO_REPROVADO',
       'SISTEMAS OPERACIONAIS_REPROVADO',
       'TEORIA GERAL DA ADMINISTRAÇÃO_REPROVADO',
       'TEORIA GERAL DOS SISTEMAS_REPROVADO', 'ÁLGEBRA LINEAR_REPROVADO',
       'ÉTICA_REPROVADO'],
      dtype='object', length=378)


# Selecionando e Organizando Colunas 

Seleção das colunas originais.

In [340]:
colunas_ordenadas = [#'discente',
                     'ano_ingresso', 'ano_nascimento', 'ano', 'sexo',
                     'estado_origem', 'cidade_origem', 'estado', 'municipio',
                    'ALGORITMOS E LÓGICA DE PROGRAMAÇÃO',
                    'ALGORITMOS E LÓGICA DE PROGRAMAÇÃO_APROVADO',
                    'ALGORITMOS E LÓGICA DE PROGRAMAÇÃO_REPROVADO',
                    'ALGORITMOS E LÓGICA DE PROGRAMAÇÃO_FALTAS',
                    'ALGORITMOS E LÓGICA DE PROGRAMAÇÃO_NOTA_1.0',
                    'ALGORITMOS E LÓGICA DE PROGRAMAÇÃO_NOTA_2.0',
                    'ALGORITMOS E LÓGICA DE PROGRAMAÇÃO_NOTA_3.0',
                    'ALGORITMOS E LÓGICA DE PROGRAMAÇÃO_MEDIA_FINAL_1.0',
                    'ALGORITMOS E LÓGICA DE PROGRAMAÇÃO_MEDIA_FINAL_2.0',
                    'ALGORITMOS E LÓGICA DE PROGRAMAÇÃO_CH_TOTAL',
                    'ALGORITMOS E LÓGICA DE PROGRAMAÇÃO_REPOSICAO',
                    'INTRODUÇÃO À INFORMÁTICA',
                    'INTRODUÇÃO À INFORMÁTICA_APROVADO',
                    'INTRODUÇÃO À INFORMÁTICA_REPROVADO',
                    'INTRODUÇÃO À INFORMÁTICA_FALTAS',
                    'INTRODUÇÃO À INFORMÁTICA_NOTA_1.0',
                    'INTRODUÇÃO À INFORMÁTICA_NOTA_2.0',
                    'INTRODUÇÃO À INFORMÁTICA_NOTA_3.0',
                    'INTRODUÇÃO À INFORMÁTICA_MEDIA_FINAL_1.0',
                    'INTRODUÇÃO À INFORMÁTICA_MEDIA_FINAL_2.0',
                    'INTRODUÇÃO À INFORMÁTICA_CH_TOTAL',
                    'INTRODUÇÃO À INFORMÁTICA_REPOSICAO',
                    'FUNDAMENTOS DE MATEMÁTICA',
                    'FUNDAMENTOS DE MATEMÁTICA_APROVADO',
                    'FUNDAMENTOS DE MATEMÁTICA_REPROVADO',
                    'FUNDAMENTOS DE MATEMÁTICA_FALTAS',
                    'FUNDAMENTOS DE MATEMÁTICA_NOTA_1.0',
                    'FUNDAMENTOS DE MATEMÁTICA_NOTA_2.0',
                    'FUNDAMENTOS DE MATEMÁTICA_NOTA_3.0',
                    'FUNDAMENTOS DE MATEMÁTICA_MEDIA_FINAL_1.0',
                    'FUNDAMENTOS DE MATEMÁTICA_MEDIA_FINAL_2.0',
                    'FUNDAMENTOS DE MATEMÁTICA_CH_TOTAL',
                    'FUNDAMENTOS DE MATEMÁTICA_REPOSICAO',
                    'LÓGICA',
                    'LÓGICA_APROVADO',
                    'LÓGICA_REPROVADO',
                    'LÓGICA_FALTAS',
                    'LÓGICA_NOTA_1.0',
                    'LÓGICA_NOTA_2.0',
                    'LÓGICA_NOTA_3.0',
                    'LÓGICA_MEDIA_FINAL_1.0',
                    'LÓGICA_MEDIA_FINAL_2.0',
                    'LÓGICA_CH_TOTAL',
                    'LÓGICA_REPOSICAO',
                    'TEORIA GERAL DA ADMINISTRAÇÃO',
                    'TEORIA GERAL DA ADMINISTRAÇÃO_APROVADO',
                    'TEORIA GERAL DA ADMINISTRAÇÃO_REPROVADO',
                    'TEORIA GERAL DA ADMINISTRAÇÃO_FALTAS',
                    'TEORIA GERAL DA ADMINISTRAÇÃO_NOTA_1.0',
                    'TEORIA GERAL DA ADMINISTRAÇÃO_NOTA_2.0',
                    'TEORIA GERAL DA ADMINISTRAÇÃO_NOTA_3.0',
                    'TEORIA GERAL DA ADMINISTRAÇÃO_MEDIA_FINAL_1.0',
                    'TEORIA GERAL DA ADMINISTRAÇÃO_MEDIA_FINAL_2.0',
                    'TEORIA GERAL DA ADMINISTRAÇÃO_CH_TOTAL',
                    'TEORIA GERAL DA ADMINISTRAÇÃO_REPOSICAO',
                    'PROGRAMAÇÃO',
                    'PROGRAMAÇÃO_APROVADO',
                    'PROGRAMAÇÃO_REPROVADO',
                    'PROGRAMAÇÃO_FALTAS',
                    'PROGRAMAÇÃO_NOTA_1.0',
                    'PROGRAMAÇÃO_NOTA_2.0',
                    'PROGRAMAÇÃO_NOTA_3.0',
                    'PROGRAMAÇÃO_MEDIA_FINAL_1.0',
                    'PROGRAMAÇÃO_MEDIA_FINAL_2.0',
                    'PROGRAMAÇÃO_CH_TOTAL',
                    'PROGRAMAÇÃO_REPOSICAO',
                    'CÁLCULO DIFERENCIAL E INTEGRAL',
                    'CÁLCULO DIFERENCIAL E INTEGRAL_APROVADO',
                    'CÁLCULO DIFERENCIAL E INTEGRAL_REPROVADO',
                    'CÁLCULO DIFERENCIAL E INTEGRAL_FALTAS',
                    'CÁLCULO DIFERENCIAL E INTEGRAL_NOTA_1.0',
                    'CÁLCULO DIFERENCIAL E INTEGRAL_NOTA_2.0',
                    'CÁLCULO DIFERENCIAL E INTEGRAL_NOTA_3.0',
                    'CÁLCULO DIFERENCIAL E INTEGRAL_MEDIA_FINAL_1.0',
                    'CÁLCULO DIFERENCIAL E INTEGRAL_MEDIA_FINAL_2.0',
                    'CÁLCULO DIFERENCIAL E INTEGRAL_CH_TOTAL',
                    'CÁLCULO DIFERENCIAL E INTEGRAL_REPOSICAO',
                    'TEORIA GERAL DOS SISTEMAS',
                    'TEORIA GERAL DOS SISTEMAS_APROVADO',
                    'TEORIA GERAL DOS SISTEMAS_REPROVADO',
                    'TEORIA GERAL DOS SISTEMAS_FALTAS',
                    'TEORIA GERAL DOS SISTEMAS_NOTA_1.0',
                    'TEORIA GERAL DOS SISTEMAS_NOTA_2.0',
                    'TEORIA GERAL DOS SISTEMAS_NOTA_3.0',
                    'TEORIA GERAL DOS SISTEMAS_MEDIA_FINAL_1.0',
                    'TEORIA GERAL DOS SISTEMAS_MEDIA_FINAL_2.0',
                    'TEORIA GERAL DOS SISTEMAS_CH_TOTAL',
                    'TEORIA GERAL DOS SISTEMAS_REPOSICAO',
                    'PROGRAMAÇÃO ORIENTADA A OBJETOS I',
                    'PROGRAMAÇÃO ORIENTADA A OBJETOS I_APROVADO',
                    'PROGRAMAÇÃO ORIENTADA A OBJETOS I_REPROVADO',
                    'PROGRAMAÇÃO ORIENTADA A OBJETOS I_FALTAS',
                    'PROGRAMAÇÃO ORIENTADA A OBJETOS I_NOTA_1.0',
                    'PROGRAMAÇÃO ORIENTADA A OBJETOS I_NOTA_2.0',
                    'PROGRAMAÇÃO ORIENTADA A OBJETOS I_NOTA_3.0',
                    'PROGRAMAÇÃO ORIENTADA A OBJETOS I_MEDIA_FINAL_1.0',
                    'PROGRAMAÇÃO ORIENTADA A OBJETOS I_MEDIA_FINAL_2.0',
                    'PROGRAMAÇÃO ORIENTADA A OBJETOS I_CH_TOTAL',
                    'PROGRAMAÇÃO ORIENTADA A OBJETOS I_REPOSICAO',
                    'ESTRUTURA DE DADOS',
                    'ESTRUTURA DE DADOS_APROVADO',
                    'ESTRUTURA DE DADOS_REPROVADO',
                    'ESTRUTURA DE DADOS_FALTAS',
                    'ESTRUTURA DE DADOS_NOTA_1.0',
                    'ESTRUTURA DE DADOS_NOTA_2.0',
                    'ESTRUTURA DE DADOS_NOTA_3.0',
                    'ESTRUTURA DE DADOS_MEDIA_FINAL_1.0',
                    'ESTRUTURA DE DADOS_MEDIA_FINAL_2.0',
                    'ESTRUTURA DE DADOS_CH_TOTAL',
                    'ESTRUTURA DE DADOS_REPOSICAO',
                    'ÁLGEBRA LINEAR',
                    'ÁLGEBRA LINEAR_APROVADO',
                    'ÁLGEBRA LINEAR_REPROVADO',
                    'ÁLGEBRA LINEAR_FALTAS',
                    'ÁLGEBRA LINEAR_NOTA_1.0',
                    'ÁLGEBRA LINEAR_NOTA_2.0',
                    'ÁLGEBRA LINEAR_NOTA_3.0',
                    'ÁLGEBRA LINEAR_MEDIA_FINAL_1.0',
                    'ÁLGEBRA LINEAR_MEDIA_FINAL_2.0',
                    'ÁLGEBRA LINEAR_CH_TOTAL',
                    'ÁLGEBRA LINEAR_REPOSICAO',
                    'ORGANIZAÇÃO, SISTEMAS E MÉTODOS',
                    'ORGANIZAÇÃO, SISTEMAS E MÉTODOS_APROVADO',
                    'ORGANIZAÇÃO, SISTEMAS E MÉTODOS_REPROVADO',
                    'ORGANIZAÇÃO, SISTEMAS E MÉTODOS_FALTAS',
                    'ORGANIZAÇÃO, SISTEMAS E MÉTODOS_NOTA_1.0',
                    'ORGANIZAÇÃO, SISTEMAS E MÉTODOS_NOTA_2.0',
                    'ORGANIZAÇÃO, SISTEMAS E MÉTODOS_NOTA_3.0',
                    'ORGANIZAÇÃO, SISTEMAS E MÉTODOS_MEDIA_FINAL_1.0',
                    'ORGANIZAÇÃO, SISTEMAS E MÉTODOS_MEDIA_FINAL_2.0',
                    'ORGANIZAÇÃO, SISTEMAS E MÉTODOS_CH_TOTAL',
                    'ORGANIZAÇÃO, SISTEMAS E MÉTODOS_REPOSICAO',
                    'FUNDAMENTOS DE SISTEMAS DE INFORMAÇÃO',
                    'FUNDAMENTOS DE SISTEMAS DE INFORMAÇÃO_APROVADO',
                    'FUNDAMENTOS DE SISTEMAS DE INFORMAÇÃO_REPROVADO',
                    'FUNDAMENTOS DE SISTEMAS DE INFORMAÇÃO_FALTAS',
                    'FUNDAMENTOS DE SISTEMAS DE INFORMAÇÃO_NOTA_1.0',
                    'FUNDAMENTOS DE SISTEMAS DE INFORMAÇÃO_NOTA_2.0',
                    'FUNDAMENTOS DE SISTEMAS DE INFORMAÇÃO_NOTA_3.0',
                    'FUNDAMENTOS DE SISTEMAS DE INFORMAÇÃO_MEDIA_FINAL_1.0',
                    'FUNDAMENTOS DE SISTEMAS DE INFORMAÇÃO_MEDIA_FINAL_2.0',
                    'FUNDAMENTOS DE SISTEMAS DE INFORMAÇÃO_CH_TOTAL',
                    'FUNDAMENTOS DE SISTEMAS DE INFORMAÇÃO_REPOSICAO',
                    'PROGRAMAÇÃO WEB',
                    'PROGRAMAÇÃO WEB_APROVADO',
                    'PROGRAMAÇÃO WEB_REPROVADO',
                    'PROGRAMAÇÃO WEB_FALTAS',
                    'PROGRAMAÇÃO WEB_NOTA_1.0',
                    'PROGRAMAÇÃO WEB_NOTA_2.0',
                    'PROGRAMAÇÃO WEB_NOTA_3.0',
                    'PROGRAMAÇÃO WEB_MEDIA_FINAL_1.0',
                    'PROGRAMAÇÃO WEB_MEDIA_FINAL_2.0',
                    'PROGRAMAÇÃO WEB_CH_TOTAL',
                    'PROGRAMAÇÃO WEB_REPOSICAO',
                    'ARQUITETURA DE COMPUTADORES',
                    'ARQUITETURA DE COMPUTADORES_APROVADO',
                    'ARQUITETURA DE COMPUTADORES_REPROVADO',
                    'ARQUITETURA DE COMPUTADORES_FALTAS',
                    'ARQUITETURA DE COMPUTADORES_NOTA_1.0',
                    'ARQUITETURA DE COMPUTADORES_NOTA_2.0',
                    'ARQUITETURA DE COMPUTADORES_NOTA_3.0',
                    'ARQUITETURA DE COMPUTADORES_MEDIA_FINAL_1.0',
                    'ARQUITETURA DE COMPUTADORES_MEDIA_FINAL_2.0',
                    'ARQUITETURA DE COMPUTADORES_CH_TOTAL',
                    'ARQUITETURA DE COMPUTADORES_REPOSICAO',
                    'PROBABILIDADE E ESTATÍSTICA',
                    'PROBABILIDADE E ESTATÍSTICA_APROVADO',
                    'PROBABILIDADE E ESTATÍSTICA_REPROVADO',
                    'PROBABILIDADE E ESTATÍSTICA_FALTAS',
                    'PROBABILIDADE E ESTATÍSTICA_NOTA_1.0',
                    'PROBABILIDADE E ESTATÍSTICA_NOTA_2.0',
                    'PROBABILIDADE E ESTATÍSTICA_NOTA_3.0',
                    'PROBABILIDADE E ESTATÍSTICA_MEDIA_FINAL_1.0',
                    'PROBABILIDADE E ESTATÍSTICA_MEDIA_FINAL_2.0',
                    'PROBABILIDADE E ESTATÍSTICA_CH_TOTAL',
                    'PROBABILIDADE E ESTATÍSTICA_REPOSICAO',
                    'BANCO DE DADOS',
                    'BANCO DE DADOS_APROVADO',
                    'BANCO DE DADOS_REPROVADO',
                    'BANCO DE DADOS_FALTAS',
                    'BANCO DE DADOS_NOTA_1.0',
                    'BANCO DE DADOS_NOTA_2.0',
                    'BANCO DE DADOS_NOTA_3.0',
                    'BANCO DE DADOS_MEDIA_FINAL_1.0',
                    'BANCO DE DADOS_MEDIA_FINAL_2.0',
                    'BANCO DE DADOS_CH_TOTAL',
                    'BANCO DE DADOS_REPOSICAO',
                    'ENGENHARIA DE SOFTWARE I',
                    'ENGENHARIA DE SOFTWARE I_APROVADO',
                    'ENGENHARIA DE SOFTWARE I_REPROVADO',
                    'ENGENHARIA DE SOFTWARE I_FALTAS',
                    'ENGENHARIA DE SOFTWARE I_NOTA_1.0',
                    'ENGENHARIA DE SOFTWARE I_NOTA_2.0',
                    'ENGENHARIA DE SOFTWARE I_NOTA_3.0',
                    'ENGENHARIA DE SOFTWARE I_MEDIA_FINAL_1.0',
                    'ENGENHARIA DE SOFTWARE I_MEDIA_FINAL_2.0',
                    'ENGENHARIA DE SOFTWARE I_CH_TOTAL',
                    'ENGENHARIA DE SOFTWARE I_REPOSICAO',
                    'PROGRAMAÇÃO ORIENTADA A OBJETOS II',
                    'PROGRAMAÇÃO ORIENTADA A OBJETOS II_APROVADO',
                    'PROGRAMAÇÃO ORIENTADA A OBJETOS II_REPROVADO',
                    'PROGRAMAÇÃO ORIENTADA A OBJETOS II_FALTAS',
                    'PROGRAMAÇÃO ORIENTADA A OBJETOS II_NOTA_1.0',
                    'PROGRAMAÇÃO ORIENTADA A OBJETOS II_NOTA_2.0',
                    'PROGRAMAÇÃO ORIENTADA A OBJETOS II_NOTA_3.0',
                    'PROGRAMAÇÃO ORIENTADA A OBJETOS II_MEDIA_FINAL_1.0',
                    'PROGRAMAÇÃO ORIENTADA A OBJETOS II_MEDIA_FINAL_2.0',
                    'PROGRAMAÇÃO ORIENTADA A OBJETOS II_CH_TOTAL',
                    'PROGRAMAÇÃO ORIENTADA A OBJETOS II_REPOSICAO',
                    'SISTEMAS OPERACIONAIS',
                    'SISTEMAS OPERACIONAIS_APROVADO',
                    'SISTEMAS OPERACIONAIS_REPROVADO',
                    'SISTEMAS OPERACIONAIS_FALTAS',
                    'SISTEMAS OPERACIONAIS_NOTA_1.0',
                    'SISTEMAS OPERACIONAIS_NOTA_2.0',
                    'SISTEMAS OPERACIONAIS_NOTA_3.0',
                    'SISTEMAS OPERACIONAIS_MEDIA_FINAL_1.0',
                    'SISTEMAS OPERACIONAIS_MEDIA_FINAL_2.0',
                    'SISTEMAS OPERACIONAIS_CH_TOTAL',
                    'SISTEMAS OPERACIONAIS_REPOSICAO',
                    'PROJETO E ADMINISTRAÇÃO DE BANCO DE DADOS',
                    'PROJETO E ADMINISTRAÇÃO DE BANCO DE DADOS_APROVADO',
                    'PROJETO E ADMINISTRAÇÃO DE BANCO DE DADOS_REPROVADO',
                    'PROJETO E ADMINISTRAÇÃO DE BANCO DE DADOS_FALTAS',
                    'PROJETO E ADMINISTRAÇÃO DE BANCO DE DADOS_NOTA_1.0',
                    'PROJETO E ADMINISTRAÇÃO DE BANCO DE DADOS_NOTA_2.0',
                    'PROJETO E ADMINISTRAÇÃO DE BANCO DE DADOS_NOTA_3.0',
                    'PROJETO E ADMINISTRAÇÃO DE BANCO DE DADOS_MEDIA_FINAL_1.0',
                    'PROJETO E ADMINISTRAÇÃO DE BANCO DE DADOS_MEDIA_FINAL_2.0',
                    'PROJETO E ADMINISTRAÇÃO DE BANCO DE DADOS_CH_TOTAL',
                    'PROJETO E ADMINISTRAÇÃO DE BANCO DE DADOS_REPOSICAO',
                    'ENGENHARIA DE SOFTWARE II',
                    'ENGENHARIA DE SOFTWARE II_APROVADO',
                    'ENGENHARIA DE SOFTWARE II_REPROVADO',
                    'ENGENHARIA DE SOFTWARE II_FALTAS',
                    'ENGENHARIA DE SOFTWARE II_NOTA_1.0',
                    'ENGENHARIA DE SOFTWARE II_NOTA_2.0',
                    'ENGENHARIA DE SOFTWARE II_NOTA_3.0',
                    'ENGENHARIA DE SOFTWARE II_MEDIA_FINAL_1.0',
                    'ENGENHARIA DE SOFTWARE II_MEDIA_FINAL_2.0',
                    'ENGENHARIA DE SOFTWARE II_CH_TOTAL',
                    'ENGENHARIA DE SOFTWARE II_REPOSICAO',
                    'REDES DE COMPUTADORES',
                    'REDES DE COMPUTADORES_APROVADO',
                    'REDES DE COMPUTADORES_REPROVADO',
                    'REDES DE COMPUTADORES_FALTAS',
                    'REDES DE COMPUTADORES_NOTA_1.0',
                    'REDES DE COMPUTADORES_NOTA_2.0',
                    'REDES DE COMPUTADORES_NOTA_3.0',
                    'REDES DE COMPUTADORES_MEDIA_FINAL_1.0',
                    'REDES DE COMPUTADORES_MEDIA_FINAL_2.0',
                    'REDES DE COMPUTADORES_CH_TOTAL',
                    'REDES DE COMPUTADORES_REPOSICAO',
                    'EMPREENDEDORISMO EM INFORMÁTICA',
                    'EMPREENDEDORISMO EM INFORMÁTICA_APROVADO',
                    'EMPREENDEDORISMO EM INFORMÁTICA_REPROVADO',
                    'EMPREENDEDORISMO EM INFORMÁTICA_FALTAS',
                    'EMPREENDEDORISMO EM INFORMÁTICA_NOTA_1.0',
                    'EMPREENDEDORISMO EM INFORMÁTICA_NOTA_2.0',
                    'EMPREENDEDORISMO EM INFORMÁTICA_NOTA_3.0',
                    'EMPREENDEDORISMO EM INFORMÁTICA_MEDIA_FINAL_1.0',
                    'EMPREENDEDORISMO EM INFORMÁTICA_MEDIA_FINAL_2.0',
                    'EMPREENDEDORISMO EM INFORMÁTICA_CH_TOTAL',
                    'EMPREENDEDORISMO EM INFORMÁTICA_REPOSICAO',
                    'GESTÃO DE PROJETO DE SOFTWARE',
                    'GESTÃO DE PROJETO DE SOFTWARE_APROVADO',
                    'GESTÃO DE PROJETO DE SOFTWARE_REPROVADO',
                    'GESTÃO DE PROJETO DE SOFTWARE_FALTAS',
                    'GESTÃO DE PROJETO DE SOFTWARE_NOTA_1.0',
                    'GESTÃO DE PROJETO DE SOFTWARE_NOTA_2.0',
                    'GESTÃO DE PROJETO DE SOFTWARE_NOTA_3.0',
                    'GESTÃO DE PROJETO DE SOFTWARE_MEDIA_FINAL_1.0',
                    'GESTÃO DE PROJETO DE SOFTWARE_MEDIA_FINAL_2.0',
                    'GESTÃO DE PROJETO DE SOFTWARE_CH_TOTAL',
                    'GESTÃO DE PROJETO DE SOFTWARE_REPOSICAO',
                    'PROGRAMAÇÃO VISUAL',
                    'PROGRAMAÇÃO VISUAL_APROVADO',
                    'PROGRAMAÇÃO VISUAL_REPROVADO',
                    'PROGRAMAÇÃO VISUAL_FALTAS',
                    'PROGRAMAÇÃO VISUAL_NOTA_1.0',
                    'PROGRAMAÇÃO VISUAL_NOTA_2.0',
                    'PROGRAMAÇÃO VISUAL_NOTA_3.0',
                    'PROGRAMAÇÃO VISUAL_MEDIA_FINAL_1.0',
                    'PROGRAMAÇÃO VISUAL_MEDIA_FINAL_2.0',
                    'PROGRAMAÇÃO VISUAL_CH_TOTAL',
                    'PROGRAMAÇÃO VISUAL_REPOSICAO',
                    'MATEMÁTICA FINANCEIRA',
                    'MATEMÁTICA FINANCEIRA_APROVADO',
                    'MATEMÁTICA FINANCEIRA_REPROVADO',
                    'MATEMÁTICA FINANCEIRA_FALTAS',
                    'MATEMÁTICA FINANCEIRA_NOTA_1.0',
                    'MATEMÁTICA FINANCEIRA_NOTA_2.0',
                    'MATEMÁTICA FINANCEIRA_NOTA_3.0',
                    'MATEMÁTICA FINANCEIRA_MEDIA_FINAL_1.0',
                    'MATEMÁTICA FINANCEIRA_MEDIA_FINAL_2.0',
                    'MATEMÁTICA FINANCEIRA_CH_TOTAL',
                    'MATEMÁTICA FINANCEIRA_REPOSICAO',
                    'SISTEMAS DE APOIO À DECISÃO',
                    'SISTEMAS DE APOIO À DECISÃO_APROVADO',
                    'SISTEMAS DE APOIO À DECISÃO_REPROVADO',
                    'SISTEMAS DE APOIO À DECISÃO_FALTAS',
                    'SISTEMAS DE APOIO À DECISÃO_NOTA_1.0',
                    'SISTEMAS DE APOIO À DECISÃO_NOTA_2.0',
                    'SISTEMAS DE APOIO À DECISÃO_NOTA_3.0',
                    'SISTEMAS DE APOIO À DECISÃO_MEDIA_FINAL_1.0',
                    'SISTEMAS DE APOIO À DECISÃO_MEDIA_FINAL_2.0',
                    'SISTEMAS DE APOIO À DECISÃO_CH_TOTAL',
                    'SISTEMAS DE APOIO À DECISÃO_REPOSICAO',
                    'ÉTICA',
                    'ÉTICA_APROVADO',
                    'ÉTICA_REPROVADO',
                    'ÉTICA_FALTAS',
                    'ÉTICA_NOTA_1.0',
                    'ÉTICA_NOTA_2.0',
                    'ÉTICA_NOTA_3.0',
                    'ÉTICA_MEDIA_FINAL_1.0',
                    'ÉTICA_MEDIA_FINAL_2.0',
                    'ÉTICA_CH_TOTAL',
                    'ÉTICA_REPOSICAO',
                    'status'
                      ]
df_merged_7 = df_merged_7[colunas_ordenadas]

In [341]:
df_merged_7.to_csv('df_ORIGINAL_328.csv', index=False, sep=',', quoting=csv.QUOTE_NONNUMERIC)

In [342]:
# Identificando as colunas que devem ser removidas com base no final do nome
colunas_remover = [col for col in df_merged_7.columns if 
                   col.endswith('_FALTAS') or 
                   col.endswith('_NOTA_1.0') or 
                   col.endswith('_NOTA_2.0') or 
                   col.endswith('_NOTA_3.0') or 
                   col.endswith('_MEDIA_FINAL_1.0') or 
                   col.endswith('_REPOSICAO')]

# Removendo as colunas identificadas
df_merged_7 = df_merged_7.drop(columns=colunas_remover)

In [343]:
# Lista de colunas a serem removidas
colunas_para_remover = [
    'ano_nascimento',
    'sexo',
    'estado_origem',
    'cidade_origem',
    'estado',
    'municipio'
]
df_merged_7 = df_merged_7.drop(columns=colunas_para_remover)

In [344]:
df_merged_7

,ano_ingresso,ano,ALGORITMOS E LÓGICA DE PROGRAMAÇÃO,ALGORITMOS E LÓGICA DE PROGRAMAÇÃO_APROVADO,ALGORITMOS E LÓGICA DE PROGRAMAÇÃO_REPROVADO,ALGORITMOS E LÓGICA DE PROGRAMAÇÃO_MEDIA_FINAL_2.0,ALGORITMOS E LÓGICA DE PROGRAMAÇÃO_CH_TOTAL,INTRODUÇÃO À INFORMÁTICA,INTRODUÇÃO À INFORMÁTICA_APROVADO,INTRODUÇÃO À INFORMÁTICA_REPROVADO,INTRODUÇÃO À INFORMÁTICA_MEDIA_FINAL_2.0,INTRODUÇÃO À INFORMÁTICA_CH_TOTAL,FUNDAMENTOS DE MATEMÁTICA,FUNDAMENTOS DE MATEMÁTICA_APROVADO,FUNDAMENTOS DE MATEMÁTICA_REPROVADO,FUNDAMENTOS DE MATEMÁTICA_MEDIA_FINAL_2.0,FUNDAMENTOS DE MATEMÁTICA_CH_TOTAL,LÓGICA,LÓGICA_APROVADO,LÓGICA_REPROVADO,LÓGICA_MEDIA_FINAL_2.0,LÓGICA_CH_TOTAL,TEORIA GERAL DA ADMINISTRAÇÃO,TEORIA GERAL DA ADMINISTRAÇÃO_APROVADO,TEORIA GERAL DA ADMINISTRAÇÃO_REPROVADO,TEORIA GERAL DA ADMINISTRAÇÃO_MEDIA_FINAL_2.0,TEORIA GERAL DA ADMINISTRAÇÃO_CH_TOTAL,PROGRAMAÇÃO,PROGRAMAÇÃO_APROVADO,PROGRAMAÇÃO_REPROVADO,PROGRAMAÇÃO_MEDIA_FINAL_2.0,PROGRAMAÇÃO_CH_TOTAL,CÁLCULO DIFERENCIAL E INTEGRAL,CÁLCULO DIFERENCIAL E INTEGRAL_APROVADO,CÁLCULO DIFERENCIAL E INTEGRAL_REPROVADO,CÁLCULO DIFERENCIAL E INTEGRAL_MEDIA_FINAL_2.0,CÁLCULO DIFERENCIAL E INTEGRAL_CH_TOTAL,TEORIA GERAL DOS SISTEMAS,TEORIA GERAL DOS SISTEMAS_APROVADO,TEORIA GERAL DOS SISTEMAS_REPROVADO,TEORIA GERAL DOS SISTEMAS_MEDIA_FINAL_2.0,TEORIA GERAL DOS SISTEMAS_CH_TOTAL,PROGRAMAÇÃO ORIENTADA A OBJETOS I,PROGRAMAÇÃO ORIENTADA A OBJETOS I_APROVADO,PROGRAMAÇÃO ORIENTADA A OBJETOS I_REPROVADO,PROGRAMAÇÃO ORIENTADA A OBJETOS I_MEDIA_FINAL_2.0,PROGRAMAÇÃO ORIENTADA A OBJETOS I_CH_TOTAL,ESTRUTURA DE DADOS,ESTRUTURA DE DADOS_APROVADO,ESTRUTURA DE DADOS_REPROVADO,ESTRUTURA DE DADOS_MEDIA_FINAL_2.0,ESTRUTURA DE DADOS_CH_TOTAL,ÁLGEBRA LINEAR,ÁLGEBRA LINEAR_APROVADO,ÁLGEBRA LINEAR_REPROVADO,ÁLGEBRA LINEAR_MEDIA_FINAL_2.0,ÁLGEBRA LINEAR_CH_TOTAL,"ORGANIZAÇÃO, SISTEMAS E MÉTODOS","ORGANIZAÇÃO, SISTEMAS E MÉTODOS_APROVADO","ORGANIZAÇÃO, SISTEMAS E MÉTODOS_REPROVADO","ORGANIZAÇÃO, SISTEMAS E MÉTODOS_MEDIA_FINAL_2.0","ORGANIZAÇÃO, SISTEMAS E MÉTODOS_CH_TOTAL",FUNDAMENTOS DE SISTEMAS DE INFORMAÇÃO,FUNDAMENTOS DE SISTEMAS DE INFORMAÇÃO_APROVADO,FUNDAMENTOS DE SISTEMAS DE INFORMAÇÃO_REPROVADO,FUNDAMENTOS DE SISTEMAS DE INFORMAÇÃO_MEDIA_FINAL_2.0,FUNDAMENTOS DE SISTEMAS DE INFORMAÇÃO_CH_TOTAL,PROGRAMAÇÃO WEB,PROGRAMAÇÃO WEB_APROVADO,PROGRAMAÇÃO WEB_REPROVADO,PROGRAMAÇÃO WEB_MEDIA_FINAL_2.0,PROGRAMAÇÃO WEB_CH_TOTAL,ARQUITETURA DE COMPUTADORES,ARQUITETURA DE COMPUTADORES_APROVADO,ARQUITETURA DE COMPUTADORES_REPROVADO,ARQUITETURA DE COMPUTADORES_MEDIA_FINAL_2.0,ARQUITETURA DE COMPUTADORES_CH_TOTAL,PROBABILIDADE E ESTATÍSTICA,PROBABILIDADE E ESTATÍSTICA_APROVADO,PROBABILIDADE E ESTATÍSTICA_REPROVADO,PROBABILIDADE E ESTATÍSTICA_MEDIA_FINAL_2.0,PROBABILIDADE E ESTATÍSTICA_CH_TOTAL,BANCO DE DADOS,BANCO DE DADOS_APROVADO,BANCO DE DADOS_REPROVADO,BANCO DE DADOS_MEDIA_FINAL_2.0,BANCO DE DADOS_CH_TOTAL,ENGENHARIA DE SOFTWARE I,ENGENHARIA DE SOFTWARE I_APROVADO,ENGENHARIA DE SOFTWARE I_REPROVADO,ENGENHARIA DE SOFTWARE I_MEDIA_FINAL_2.0,ENGENHARIA DE SOFTWARE I_CH_TOTAL,PROGRAMAÇÃO ORIENTADA A OBJETOS II,PROGRAMAÇÃO ORIENTADA A OBJETOS II_APROVADO,PROGRAMAÇÃO ORIENTADA A OBJETOS II_REPROVADO,PROGRAMAÇÃO ORIENTADA A OBJETOS II_MEDIA_FINAL_2.0,PROGRAMAÇÃO ORIENTADA A OBJETOS II_CH_TOTAL,SISTEMAS OPERACIONAIS,SISTEMAS OPERACIONAIS_APROVADO,SISTEMAS OPERACIONAIS_REPROVADO,SISTEMAS OPERACIONAIS_MEDIA_FINAL_2.0,SISTEMAS OPERACIONAIS_CH_TOTAL,PROJETO E ADMINISTRAÇÃO DE BANCO DE DADOS,PROJETO E ADMINISTRAÇÃO DE BANCO DE DADOS_APROVADO,PROJETO E ADMINISTRAÇÃO DE BANCO DE DADOS_REPROVADO,PROJETO E ADMINISTRAÇÃO DE BANCO DE DADOS_MEDIA_FINAL_2.0,PROJETO E ADMINISTRAÇÃO DE BANCO DE DADOS_CH_TOTAL,ENGENHARIA DE SOFTWARE II,ENGENHARIA DE SOFTWARE II_APROVADO,ENGENHARIA DE SOFTWARE II_REPROVADO,ENGENHARIA DE SOFTWARE II_MEDIA_FINAL_2.0,ENGENHARIA DE SOFTWARE II_CH_TOTAL,REDES DE COMPUTADORES,REDES DE COMPUTADORES_APROVADO,REDES DE COMPUTADORES_REPROVADO,REDES DE COMPUTADORES_MEDIA_FINAL_2.0,REDES DE COMPUTADORES_CH_TOTAL,EMPREENDEDORISM

In [345]:
df_merged_7.to_csv('df_ORIGINAL_148.csv', index=False, sep=',', quoting=csv.QUOTE_NONNUMERIC)

Seleção das colunas usando as que foram criadas e as originais.

In [346]:
colunas_ordenadas = ['discente','ano_ingresso', 'ano_nascimento', 'ano', 'sexo',
                     'estado_origem', 'cidade_origem', 'estado', 'municipio',
                    'ALGORITMOS E LÓGICA DE PROGRAMAÇÃO',
                    'ALGORITMOS E LÓGICA DE PROGRAMAÇÃO_SEMESTRE',
                    'ALGORITMOS E LÓGICA DE PROGRAMAÇÃO_APROVADO',
                    'ALGORITMOS E LÓGICA DE PROGRAMAÇÃO_REPROVADO',
                    'ALGORITMOS E LÓGICA DE PROGRAMAÇÃO_FALTAS',
                    'ALGORITMOS E LÓGICA DE PROGRAMAÇÃO_NOTA_1.0',
                    'ALGORITMOS E LÓGICA DE PROGRAMAÇÃO_NOTA_2.0',
                    'ALGORITMOS E LÓGICA DE PROGRAMAÇÃO_NOTA_3.0',
                    'ALGORITMOS E LÓGICA DE PROGRAMAÇÃO_MEDIA_FINAL_1.0',
                    'ALGORITMOS E LÓGICA DE PROGRAMAÇÃO_MEDIA_FINAL_2.0',
                    'ALGORITMOS E LÓGICA DE PROGRAMAÇÃO_CH_TOTAL',
                    'ALGORITMOS E LÓGICA DE PROGRAMAÇÃO_REPOSICAO',
                    'INTRODUÇÃO À INFORMÁTICA',
                    'INTRODUÇÃO À INFORMÁTICA_SEMESTRE',
                    'INTRODUÇÃO À INFORMÁTICA_APROVADO',
                    'INTRODUÇÃO À INFORMÁTICA_REPROVADO',
                    'INTRODUÇÃO À INFORMÁTICA_FALTAS',
                    'INTRODUÇÃO À INFORMÁTICA_NOTA_1.0',
                    'INTRODUÇÃO À INFORMÁTICA_NOTA_2.0',
                    'INTRODUÇÃO À INFORMÁTICA_NOTA_3.0',
                    'INTRODUÇÃO À INFORMÁTICA_MEDIA_FINAL_1.0',
                    'INTRODUÇÃO À INFORMÁTICA_MEDIA_FINAL_2.0',
                    'INTRODUÇÃO À INFORMÁTICA_CH_TOTAL',
                    'INTRODUÇÃO À INFORMÁTICA_REPOSICAO',
                    'FUNDAMENTOS DE MATEMÁTICA',
                    'FUNDAMENTOS DE MATEMÁTICA_SEMESTRE',
                    'FUNDAMENTOS DE MATEMÁTICA_APROVADO',
                    'FUNDAMENTOS DE MATEMÁTICA_REPROVADO',
                    'FUNDAMENTOS DE MATEMÁTICA_FALTAS',
                    'FUNDAMENTOS DE MATEMÁTICA_NOTA_1.0',
                    'FUNDAMENTOS DE MATEMÁTICA_NOTA_2.0',
                    'FUNDAMENTOS DE MATEMÁTICA_NOTA_3.0',
                    'FUNDAMENTOS DE MATEMÁTICA_MEDIA_FINAL_1.0',
                    'FUNDAMENTOS DE MATEMÁTICA_MEDIA_FINAL_2.0',
                    'FUNDAMENTOS DE MATEMÁTICA_CH_TOTAL',
                    'FUNDAMENTOS DE MATEMÁTICA_REPOSICAO',
                    'LÓGICA',
                    'LÓGICA_SEMESTRE',
                    'LÓGICA_APROVADO',
                    'LÓGICA_REPROVADO',
                    'LÓGICA_FALTAS',
                    'LÓGICA_NOTA_1.0',
                    'LÓGICA_NOTA_2.0',
                    'LÓGICA_NOTA_3.0',
                    'LÓGICA_MEDIA_FINAL_1.0',
                    'LÓGICA_MEDIA_FINAL_2.0',
                    'LÓGICA_CH_TOTAL',
                    'LÓGICA_REPOSICAO',
                    'TEORIA GERAL DA ADMINISTRAÇÃO',
                    'TEORIA GERAL DA ADMINISTRAÇÃO_SEMESTRE',
                    'TEORIA GERAL DA ADMINISTRAÇÃO_APROVADO',
                    'TEORIA GERAL DA ADMINISTRAÇÃO_REPROVADO',
                    'TEORIA GERAL DA ADMINISTRAÇÃO_FALTAS',
                    'TEORIA GERAL DA ADMINISTRAÇÃO_NOTA_1.0',
                    'TEORIA GERAL DA ADMINISTRAÇÃO_NOTA_2.0',
                    'TEORIA GERAL DA ADMINISTRAÇÃO_NOTA_3.0',
                    'TEORIA GERAL DA ADMINISTRAÇÃO_MEDIA_FINAL_1.0',
                    'TEORIA GERAL DA ADMINISTRAÇÃO_MEDIA_FINAL_2.0',
                    'TEORIA GERAL DA ADMINISTRAÇÃO_CH_TOTAL',
                    'TEORIA GERAL DA ADMINISTRAÇÃO_REPOSICAO',
                    'PROGRAMAÇÃO',
                    'PROGRAMAÇÃO_SEMESTRE',
                    'PROGRAMAÇÃO_APROVADO',
                    'PROGRAMAÇÃO_REPROVADO',
                    'PROGRAMAÇÃO_FALTAS',
                    'PROGRAMAÇÃO_NOTA_1.0',
                    'PROGRAMAÇÃO_NOTA_2.0',
                    'PROGRAMAÇÃO_NOTA_3.0',
                    'PROGRAMAÇÃO_MEDIA_FINAL_1.0',
                    'PROGRAMAÇÃO_MEDIA_FINAL_2.0',
                    'PROGRAMAÇÃO_CH_TOTAL',
                    'PROGRAMAÇÃO_REPOSICAO',
                    'CÁLCULO DIFERENCIAL E INTEGRAL',
                    'CÁLCULO DIFERENCIAL E INTEGRAL_SEMESTRE',
                    'CÁLCULO DIFERENCIAL E INTEGRAL_APROVADO',
                    'CÁLCULO DIFERENCIAL E INTEGRAL_REPROVADO',
                    'CÁLCULO DIFERENCIAL E INTEGRAL_FALTAS',
                    'CÁLCULO DIFERENCIAL E INTEGRAL_NOTA_1.0',
                    'CÁLCULO DIFERENCIAL E INTEGRAL_NOTA_2.0',
                    'CÁLCULO DIFERENCIAL E INTEGRAL_NOTA_3.0',
                    'CÁLCULO DIFERENCIAL E INTEGRAL_MEDIA_FINAL_1.0',
                    'CÁLCULO DIFERENCIAL E INTEGRAL_MEDIA_FINAL_2.0',
                    'CÁLCULO DIFERENCIAL E INTEGRAL_CH_TOTAL',
                    'CÁLCULO DIFERENCIAL E INTEGRAL_REPOSICAO',
                    'TEORIA GERAL DOS SISTEMAS',
                    'TEORIA GERAL DOS SISTEMAS_SEMESTRE',
                    'TEORIA GERAL DOS SISTEMAS_APROVADO',
                    'TEORIA GERAL DOS SISTEMAS_REPROVADO',
                    'TEORIA GERAL DOS SISTEMAS_FALTAS',
                    'TEORIA GERAL DOS SISTEMAS_NOTA_1.0',
                    'TEORIA GERAL DOS SISTEMAS_NOTA_2.0',
                    'TEORIA GERAL DOS SISTEMAS_NOTA_3.0',
                    'TEORIA GERAL DOS SISTEMAS_MEDIA_FINAL_1.0',
                    'TEORIA GERAL DOS SISTEMAS_MEDIA_FINAL_2.0',
                    'TEORIA GERAL DOS SISTEMAS_CH_TOTAL',
                    'TEORIA GERAL DOS SISTEMAS_REPOSICAO',
                    'PROGRAMAÇÃO ORIENTADA A OBJETOS I',
                    'PROGRAMAÇÃO ORIENTADA A OBJETOS I_SEMESTRE',
                    'PROGRAMAÇÃO ORIENTADA A OBJETOS I_APROVADO',
                    'PROGRAMAÇÃO ORIENTADA A OBJETOS I_REPROVADO',
                    'PROGRAMAÇÃO ORIENTADA A OBJETOS I_FALTAS',
                    'PROGRAMAÇÃO ORIENTADA A OBJETOS I_NOTA_1.0',
                    'PROGRAMAÇÃO ORIENTADA A OBJETOS I_NOTA_2.0',
                    'PROGRAMAÇÃO ORIENTADA A OBJETOS I_NOTA_3.0',
                    'PROGRAMAÇÃO ORIENTADA A OBJETOS I_MEDIA_FINAL_1.0',
                    'PROGRAMAÇÃO ORIENTADA A OBJETOS I_MEDIA_FINAL_2.0',
                    'PROGRAMAÇÃO ORIENTADA A OBJETOS I_CH_TOTAL',
                    'PROGRAMAÇÃO ORIENTADA A OBJETOS I_REPOSICAO',
                    'ESTRUTURA DE DADOS',
                    'ESTRUTURA DE DADOS_SEMESTRE',
                    'ESTRUTURA DE DADOS_APROVADO',
                    'ESTRUTURA DE DADOS_REPROVADO',
                    'ESTRUTURA DE DADOS_FALTAS',
                    'ESTRUTURA DE DADOS_NOTA_1.0',
                    'ESTRUTURA DE DADOS_NOTA_2.0',
                    'ESTRUTURA DE DADOS_NOTA_3.0',
                    'ESTRUTURA DE DADOS_MEDIA_FINAL_1.0',
                    'ESTRUTURA DE DADOS_MEDIA_FINAL_2.0',
                    'ESTRUTURA DE DADOS_CH_TOTAL',
                    'ESTRUTURA DE DADOS_REPOSICAO',
                    'ÁLGEBRA LINEAR',
                    'ÁLGEBRA LINEAR_SEMESTRE',
                    'ÁLGEBRA LINEAR_APROVADO',
                    'ÁLGEBRA LINEAR_REPROVADO',
                    'ÁLGEBRA LINEAR_FALTAS',
                    'ÁLGEBRA LINEAR_NOTA_1.0',
                    'ÁLGEBRA LINEAR_NOTA_2.0',
                    'ÁLGEBRA LINEAR_NOTA_3.0',
                    'ÁLGEBRA LINEAR_MEDIA_FINAL_1.0',
                    'ÁLGEBRA LINEAR_MEDIA_FINAL_2.0',
                    'ÁLGEBRA LINEAR_CH_TOTAL',
                    'ÁLGEBRA LINEAR_REPOSICAO',
                    'ORGANIZAÇÃO, SISTEMAS E MÉTODOS',
                    'ORGANIZAÇÃO, SISTEMAS E MÉTODOS_SEMESTRE',
                    'ORGANIZAÇÃO, SISTEMAS E MÉTODOS_APROVADO',
                    'ORGANIZAÇÃO, SISTEMAS E MÉTODOS_REPROVADO',
                    'ORGANIZAÇÃO, SISTEMAS E MÉTODOS_FALTAS',
                    'ORGANIZAÇÃO, SISTEMAS E MÉTODOS_NOTA_1.0',
                    'ORGANIZAÇÃO, SISTEMAS E MÉTODOS_NOTA_2.0',
                    'ORGANIZAÇÃO, SISTEMAS E MÉTODOS_NOTA_3.0',
                    'ORGANIZAÇÃO, SISTEMAS E MÉTODOS_MEDIA_FINAL_1.0',
                    'ORGANIZAÇÃO, SISTEMAS E MÉTODOS_MEDIA_FINAL_2.0',
                    'ORGANIZAÇÃO, SISTEMAS E MÉTODOS_CH_TOTAL',
                    'ORGANIZAÇÃO, SISTEMAS E MÉTODOS_REPOSICAO',
                    'FUNDAMENTOS DE SISTEMAS DE INFORMAÇÃO',
                    'FUNDAMENTOS DE SISTEMAS DE INFORMAÇÃO_SEMESTRE',
                    'FUNDAMENTOS DE SISTEMAS DE INFORMAÇÃO_APROVADO',
                    'FUNDAMENTOS DE SISTEMAS DE INFORMAÇÃO_REPROVADO',
                    'FUNDAMENTOS DE SISTEMAS DE INFORMAÇÃO_FALTAS',
                    'FUNDAMENTOS DE SISTEMAS DE INFORMAÇÃO_NOTA_1.0',
                    'FUNDAMENTOS DE SISTEMAS DE INFORMAÇÃO_NOTA_2.0',
                    'FUNDAMENTOS DE SISTEMAS DE INFORMAÇÃO_NOTA_3.0',
                    'FUNDAMENTOS DE SISTEMAS DE INFORMAÇÃO_MEDIA_FINAL_1.0',
                    'FUNDAMENTOS DE SISTEMAS DE INFORMAÇÃO_MEDIA_FINAL_2.0',
                    'FUNDAMENTOS DE SISTEMAS DE INFORMAÇÃO_CH_TOTAL',
                    'FUNDAMENTOS DE SISTEMAS DE INFORMAÇÃO_REPOSICAO',
                    'PROGRAMAÇÃO WEB',
                    'PROGRAMAÇÃO WEB_SEMESTRE',
                    'PROGRAMAÇÃO WEB_APROVADO',
                    'PROGRAMAÇÃO WEB_REPROVADO',
                    'PROGRAMAÇÃO WEB_FALTAS',
                    'PROGRAMAÇÃO WEB_NOTA_1.0',
                    'PROGRAMAÇÃO WEB_NOTA_2.0',
                    'PROGRAMAÇÃO WEB_NOTA_3.0',
                    'PROGRAMAÇÃO WEB_MEDIA_FINAL_1.0',
                    'PROGRAMAÇÃO WEB_MEDIA_FINAL_2.0',
                    'PROGRAMAÇÃO WEB_CH_TOTAL',
                    'PROGRAMAÇÃO WEB_REPOSICAO',
                    'ARQUITETURA DE COMPUTADORES',
                    'ARQUITETURA DE COMPUTADORES_SEMESTRE',
                    'ARQUITETURA DE COMPUTADORES_APROVADO',
                    'ARQUITETURA DE COMPUTADORES_REPROVADO',
                    'ARQUITETURA DE COMPUTADORES_FALTAS',
                    'ARQUITETURA DE COMPUTADORES_NOTA_1.0',
                    'ARQUITETURA DE COMPUTADORES_NOTA_2.0',
                    'ARQUITETURA DE COMPUTADORES_NOTA_3.0',
                    'ARQUITETURA DE COMPUTADORES_MEDIA_FINAL_1.0',
                    'ARQUITETURA DE COMPUTADORES_MEDIA_FINAL_2.0',
                    'ARQUITETURA DE COMPUTADORES_CH_TOTAL',
                    'ARQUITETURA DE COMPUTADORES_REPOSICAO',
                    'PROBABILIDADE E ESTATÍSTICA',
                    'PROBABILIDADE E ESTATÍSTICA_SEMESTRE',
                    'PROBABILIDADE E ESTATÍSTICA_APROVADO',
                    'PROBABILIDADE E ESTATÍSTICA_REPROVADO',
                    'PROBABILIDADE E ESTATÍSTICA_FALTAS',
                    'PROBABILIDADE E ESTATÍSTICA_NOTA_1.0',
                    'PROBABILIDADE E ESTATÍSTICA_NOTA_2.0',
                    'PROBABILIDADE E ESTATÍSTICA_NOTA_3.0',
                    'PROBABILIDADE E ESTATÍSTICA_MEDIA_FINAL_1.0',
                    'PROBABILIDADE E ESTATÍSTICA_MEDIA_FINAL_2.0',
                    'PROBABILIDADE E ESTATÍSTICA_CH_TOTAL',
                    'PROBABILIDADE E ESTATÍSTICA_REPOSICAO',
                    'BANCO DE DADOS',
                    'BANCO DE DADOS_SEMESTRE',
                    'BANCO DE DADOS_APROVADO',
                    'BANCO DE DADOS_REPROVADO',
                    'BANCO DE DADOS_FALTAS',
                    'BANCO DE DADOS_NOTA_1.0',
                    'BANCO DE DADOS_NOTA_2.0',
                    'BANCO DE DADOS_NOTA_3.0',
                    'BANCO DE DADOS_MEDIA_FINAL_1.0',
                    'BANCO DE DADOS_MEDIA_FINAL_2.0',
                    'BANCO DE DADOS_CH_TOTAL',
                    'BANCO DE DADOS_REPOSICAO',
                    'ENGENHARIA DE SOFTWARE I',
                    'ENGENHARIA DE SOFTWARE I_SEMESTRE',
                    'ENGENHARIA DE SOFTWARE I_APROVADO',
                    'ENGENHARIA DE SOFTWARE I_REPROVADO',
                    'ENGENHARIA DE SOFTWARE I_FALTAS',
                    'ENGENHARIA DE SOFTWARE I_NOTA_1.0',
                    'ENGENHARIA DE SOFTWARE I_NOTA_2.0',
                    'ENGENHARIA DE SOFTWARE I_NOTA_3.0',
                    'ENGENHARIA DE SOFTWARE I_MEDIA_FINAL_1.0',
                    'ENGENHARIA DE SOFTWARE I_MEDIA_FINAL_2.0',
                    'ENGENHARIA DE SOFTWARE I_CH_TOTAL',
                    'ENGENHARIA DE SOFTWARE I_REPOSICAO',
                    'PROGRAMAÇÃO ORIENTADA A OBJETOS II',
                    'PROGRAMAÇÃO ORIENTADA A OBJETOS II_SEMESTRE',
                    'PROGRAMAÇÃO ORIENTADA A OBJETOS II_APROVADO',
                    'PROGRAMAÇÃO ORIENTADA A OBJETOS II_REPROVADO',
                    'PROGRAMAÇÃO ORIENTADA A OBJETOS II_FALTAS',
                    'PROGRAMAÇÃO ORIENTADA A OBJETOS II_NOTA_1.0',
                    'PROGRAMAÇÃO ORIENTADA A OBJETOS II_NOTA_2.0',
                    'PROGRAMAÇÃO ORIENTADA A OBJETOS II_NOTA_3.0',
                    'PROGRAMAÇÃO ORIENTADA A OBJETOS II_MEDIA_FINAL_1.0',
                    'PROGRAMAÇÃO ORIENTADA A OBJETOS II_MEDIA_FINAL_2.0',
                    'PROGRAMAÇÃO ORIENTADA A OBJETOS II_CH_TOTAL',
                    'PROGRAMAÇÃO ORIENTADA A OBJETOS II_REPOSICAO',
                    'SISTEMAS OPERACIONAIS',
                    'SISTEMAS OPERACIONAIS_SEMESTRE',
                    'SISTEMAS OPERACIONAIS_APROVADO',
                    'SISTEMAS OPERACIONAIS_REPROVADO',
                    'SISTEMAS OPERACIONAIS_FALTAS',
                    'SISTEMAS OPERACIONAIS_NOTA_1.0',
                    'SISTEMAS OPERACIONAIS_NOTA_2.0',
                    'SISTEMAS OPERACIONAIS_NOTA_3.0',
                    'SISTEMAS OPERACIONAIS_MEDIA_FINAL_1.0',
                    'SISTEMAS OPERACIONAIS_MEDIA_FINAL_2.0',
                    'SISTEMAS OPERACIONAIS_CH_TOTAL',
                    'SISTEMAS OPERACIONAIS_REPOSICAO',
                    'PROJETO E ADMINISTRAÇÃO DE BANCO DE DADOS',
                    'PROJETO E ADMINISTRAÇÃO DE BANCO DE DADOS_SEMESTRE',
                    'PROJETO E ADMINISTRAÇÃO DE BANCO DE DADOS_APROVADO',
                    'PROJETO E ADMINISTRAÇÃO DE BANCO DE DADOS_REPROVADO',
                    'PROJETO E ADMINISTRAÇÃO DE BANCO DE DADOS_FALTAS',
                    'PROJETO E ADMINISTRAÇÃO DE BANCO DE DADOS_NOTA_1.0',
                    'PROJETO E ADMINISTRAÇÃO DE BANCO DE DADOS_NOTA_2.0',
                    'PROJETO E ADMINISTRAÇÃO DE BANCO DE DADOS_NOTA_3.0',
                    'PROJETO E ADMINISTRAÇÃO DE BANCO DE DADOS_MEDIA_FINAL_1.0',
                    'PROJETO E ADMINISTRAÇÃO DE BANCO DE DADOS_MEDIA_FINAL_2.0',
                    'PROJETO E ADMINISTRAÇÃO DE BANCO DE DADOS_CH_TOTAL',
                    'PROJETO E ADMINISTRAÇÃO DE BANCO DE DADOS_REPOSICAO',
                    'ENGENHARIA DE SOFTWARE II',
                    'ENGENHARIA DE SOFTWARE II_SEMESTRE',
                    'ENGENHARIA DE SOFTWARE II_APROVADO',
                    'ENGENHARIA DE SOFTWARE II_REPROVADO',
                    'ENGENHARIA DE SOFTWARE II_FALTAS',
                    'ENGENHARIA DE SOFTWARE II_NOTA_1.0',
                    'ENGENHARIA DE SOFTWARE II_NOTA_2.0',
                    'ENGENHARIA DE SOFTWARE II_NOTA_3.0',
                    'ENGENHARIA DE SOFTWARE II_MEDIA_FINAL_1.0',
                    'ENGENHARIA DE SOFTWARE II_MEDIA_FINAL_2.0',
                    'ENGENHARIA DE SOFTWARE II_CH_TOTAL',
                    'ENGENHARIA DE SOFTWARE II_REPOSICAO',
                    'REDES DE COMPUTADORES',
                    'REDES DE COMPUTADORES_SEMESTRE',
                    'REDES DE COMPUTADORES_APROVADO',
                    'REDES DE COMPUTADORES_REPROVADO',
                    'REDES DE COMPUTADORES_FALTAS',
                    'REDES DE COMPUTADORES_NOTA_1.0',
                    'REDES DE COMPUTADORES_NOTA_2.0',
                    'REDES DE COMPUTADORES_NOTA_3.0',
                    'REDES DE COMPUTADORES_MEDIA_FINAL_1.0',
                    'REDES DE COMPUTADORES_MEDIA_FINAL_2.0',
                    'REDES DE COMPUTADORES_CH_TOTAL',
                    'REDES DE COMPUTADORES_REPOSICAO',
                    'EMPREENDEDORISMO EM INFORMÁTICA',
                    'EMPREENDEDORISMO EM INFORMÁTICA_SEMESTRE',
                    'EMPREENDEDORISMO EM INFORMÁTICA_APROVADO',
                    'EMPREENDEDORISMO EM INFORMÁTICA_REPROVADO',
                    'EMPREENDEDORISMO EM INFORMÁTICA_FALTAS',
                    'EMPREENDEDORISMO EM INFORMÁTICA_NOTA_1.0',
                    'EMPREENDEDORISMO EM INFORMÁTICA_NOTA_2.0',
                    'EMPREENDEDORISMO EM INFORMÁTICA_NOTA_3.0',
                    'EMPREENDEDORISMO EM INFORMÁTICA_MEDIA_FINAL_1.0',
                    'EMPREENDEDORISMO EM INFORMÁTICA_MEDIA_FINAL_2.0',
                    'EMPREENDEDORISMO EM INFORMÁTICA_CH_TOTAL',
                    'EMPREENDEDORISMO EM INFORMÁTICA_REPOSICAO',
                    'GESTÃO DE PROJETO DE SOFTWARE',
                    'GESTÃO DE PROJETO DE SOFTWARE_SEMESTRE',
                    'GESTÃO DE PROJETO DE SOFTWARE_APROVADO',
                    'GESTÃO DE PROJETO DE SOFTWARE_REPROVADO',
                    'GESTÃO DE PROJETO DE SOFTWARE_FALTAS',
                    'GESTÃO DE PROJETO DE SOFTWARE_NOTA_1.0',
                    'GESTÃO DE PROJETO DE SOFTWARE_NOTA_2.0',
                    'GESTÃO DE PROJETO DE SOFTWARE_NOTA_3.0',
                    'GESTÃO DE PROJETO DE SOFTWARE_MEDIA_FINAL_1.0',
                    'GESTÃO DE PROJETO DE SOFTWARE_MEDIA_FINAL_2.0',
                    'GESTÃO DE PROJETO DE SOFTWARE_CH_TOTAL',
                    'GESTÃO DE PROJETO DE SOFTWARE_REPOSICAO',
                    'PROGRAMAÇÃO VISUAL',
                    'PROGRAMAÇÃO VISUAL_SEMESTRE',
                    'PROGRAMAÇÃO VISUAL_APROVADO',
                    'PROGRAMAÇÃO VISUAL_REPROVADO',
                    'PROGRAMAÇÃO VISUAL_FALTAS',
                    'PROGRAMAÇÃO VISUAL_NOTA_1.0',
                    'PROGRAMAÇÃO VISUAL_NOTA_2.0',
                    'PROGRAMAÇÃO VISUAL_NOTA_3.0',
                    'PROGRAMAÇÃO VISUAL_MEDIA_FINAL_1.0',
                    'PROGRAMAÇÃO VISUAL_MEDIA_FINAL_2.0',
                    'PROGRAMAÇÃO VISUAL_CH_TOTAL',
                    'PROGRAMAÇÃO VISUAL_REPOSICAO',
                    'MATEMÁTICA FINANCEIRA',
                    'MATEMÁTICA FINANCEIRA_SEMESTRE',
                    'MATEMÁTICA FINANCEIRA_APROVADO',
                    'MATEMÁTICA FINANCEIRA_REPROVADO',
                    'MATEMÁTICA FINANCEIRA_FALTAS',
                    'MATEMÁTICA FINANCEIRA_NOTA_1.0',
                    'MATEMÁTICA FINANCEIRA_NOTA_2.0',
                    'MATEMÁTICA FINANCEIRA_NOTA_3.0',
                    'MATEMÁTICA FINANCEIRA_MEDIA_FINAL_1.0',
                    'MATEMÁTICA FINANCEIRA_MEDIA_FINAL_2.0',
                    'MATEMÁTICA FINANCEIRA_CH_TOTAL',
                    'MATEMÁTICA FINANCEIRA_REPOSICAO',
                    'SISTEMAS DE APOIO À DECISÃO',
                    'SISTEMAS DE APOIO À DECISÃO_SEMESTRE',
                    'SISTEMAS DE APOIO À DECISÃO_APROVADO',
                    'SISTEMAS DE APOIO À DECISÃO_REPROVADO',
                    'SISTEMAS DE APOIO À DECISÃO_FALTAS',
                    'SISTEMAS DE APOIO À DECISÃO_NOTA_1.0',
                    'SISTEMAS DE APOIO À DECISÃO_NOTA_2.0',
                    'SISTEMAS DE APOIO À DECISÃO_NOTA_3.0',
                    'SISTEMAS DE APOIO À DECISÃO_MEDIA_FINAL_1.0',
                    'SISTEMAS DE APOIO À DECISÃO_MEDIA_FINAL_2.0',
                    'SISTEMAS DE APOIO À DECISÃO_CH_TOTAL',
                    'SISTEMAS DE APOIO À DECISÃO_REPOSICAO',
                    'ÉTICA',
                    'ÉTICA_SEMESTRE',
                    'ÉTICA_APROVADO',
                    'ÉTICA_REPROVADO',
                    'ÉTICA_FALTAS',
                    'ÉTICA_NOTA_1.0',
                    'ÉTICA_NOTA_2.0',
                    'ÉTICA_NOTA_3.0',
                    'ÉTICA_MEDIA_FINAL_1.0',
                    'ÉTICA_MEDIA_FINAL_2.0',
                    'ÉTICA_CH_TOTAL',
                    'ÉTICA_REPOSICAO',
                    #'tempo_relativo',
                    'status'
                      ]
df_merged_7 = df_merged_7[colunas_ordenadas]

KeyError: "['discente', 'ano_nascimento', 'sexo', 'estado_origem', 'cidade_origem', 'estado', 'municipio', 'ALGORITMOS E LÓGICA DE PROGRAMAÇÃO_SEMESTRE', 'ALGORITMOS E LÓGICA DE PROGRAMAÇÃO_FALTAS', 'ALGORITMOS E LÓGICA DE PROGRAMAÇÃO_NOTA_1.0', 'ALGORITMOS E LÓGICA DE PROGRAMAÇÃO_NOTA_2.0', 'ALGORITMOS E LÓGICA DE PROGRAMAÇÃO_NOTA_3.0', 'ALGORITMOS E LÓGICA DE PROGRAMAÇÃO_MEDIA_FINAL_1.0', 'ALGORITMOS E LÓGICA DE PROGRAMAÇÃO_REPOSICAO', 'INTRODUÇÃO À INFORMÁTICA_SEMESTRE', 'INTRODUÇÃO À INFORMÁTICA_FALTAS', 'INTRODUÇÃO À INFORMÁTICA_NOTA_1.0', 'INTRODUÇÃO À INFORMÁTICA_NOTA_2.0', 'INTRODUÇÃO À INFORMÁTICA_NOTA_3.0', 'INTRODUÇÃO À INFORMÁTICA_MEDIA_FINAL_1.0', 'INTRODUÇÃO À INFORMÁTICA_REPOSICAO', 'FUNDAMENTOS DE MATEMÁTICA_SEMESTRE', 'FUNDAMENTOS DE MATEMÁTICA_FALTAS', 'FUNDAMENTOS DE MATEMÁTICA_NOTA_1.0', 'FUNDAMENTOS DE MATEMÁTICA_NOTA_2.0', 'FUNDAMENTOS DE MATEMÁTICA_NOTA_3.0', 'FUNDAMENTOS DE MATEMÁTICA_MEDIA_FINAL_1.0', 'FUNDAMENTOS DE MATEMÁTICA_REPOSICAO', 'LÓGICA_SEMESTRE', 'L

Mudando a ordem das colunas.

In [ ]:
"""colunas_ordenadas = ['discente','ano_ingresso', 'ultimo_periodo','ch_cumprida',
                    'ch_cumprida_dividida', 'ch_reprovacao',
                    'taxa_reprovacao', 'semestre_total', 'semestre_dividido', 'matricula',
                    'ALGORITMOS E LÓGICA DE PROGRAMAÇÃO',
                    'ALGORITMOS E LÓGICA DE PROGRAMAÇÃO_SEMESTRE',
                    'ALGORITMOS E LÓGICA DE PROGRAMAÇÃO_APROVADO',
                    'ALGORITMOS E LÓGICA DE PROGRAMAÇÃO_REPROVADO',
                    'ALGORITMOS E LÓGICA DE PROGRAMAÇÃO_FALTAS',
                    'ALGORITMOS E LÓGICA DE PROGRAMAÇÃO_NOTA_1.0',
                    'ALGORITMOS E LÓGICA DE PROGRAMAÇÃO_NOTA_2.0',
                    'ALGORITMOS E LÓGICA DE PROGRAMAÇÃO_NOTA_3.0',
                    'ALGORITMOS E LÓGICA DE PROGRAMAÇÃO_MEDIA_FINAL_1.0',
                    'ALGORITMOS E LÓGICA DE PROGRAMAÇÃO_MEDIA_FINAL_2.0',
                    'INTRODUÇÃO À INFORMÁTICA',
                    'INTRODUÇÃO À INFORMÁTICA_SEMESTRE',
                    'INTRODUÇÃO À INFORMÁTICA_APROVADO',
                    'INTRODUÇÃO À INFORMÁTICA_REPROVADO',
                    'INTRODUÇÃO À INFORMÁTICA_FALTAS',
                    'INTRODUÇÃO À INFORMÁTICA_NOTA_1.0',
                    'INTRODUÇÃO À INFORMÁTICA_NOTA_2.0',
                    'INTRODUÇÃO À INFORMÁTICA_NOTA_3.0',
                    'INTRODUÇÃO À INFORMÁTICA_MEDIA_FINAL_1.0',
                    'INTRODUÇÃO À INFORMÁTICA_MEDIA_FINAL_2.0',
                    'FUNDAMENTOS DE MATEMÁTICA',
                    'FUNDAMENTOS DE MATEMÁTICA_SEMESTRE',
                    'FUNDAMENTOS DE MATEMÁTICA_APROVADO',
                    'FUNDAMENTOS DE MATEMÁTICA_REPROVADO',
                    'FUNDAMENTOS DE MATEMÁTICA_FALTAS',
                    'FUNDAMENTOS DE MATEMÁTICA_NOTA_1.0',
                    'FUNDAMENTOS DE MATEMÁTICA_NOTA_2.0',
                    'FUNDAMENTOS DE MATEMÁTICA_NOTA_3.0',
                    'FUNDAMENTOS DE MATEMÁTICA_MEDIA_FINAL_1.0',
                    'FUNDAMENTOS DE MATEMÁTICA_MEDIA_FINAL_2.0',
                    'LÓGICA',
                    'LÓGICA_SEMESTRE',
                    'LÓGICA_APROVADO',
                    'LÓGICA_REPROVADO',
                    'LÓGICA_FALTAS',
                    'LÓGICA_NOTA_1.0',
                    'LÓGICA_NOTA_2.0',
                    'LÓGICA_NOTA_3.0',
                    'LÓGICA_MEDIA_FINAL_1.0',
                    'LÓGICA_MEDIA_FINAL_2.0',
                    'TEORIA GERAL DA ADMINISTRAÇÃO',
                    'TEORIA GERAL DA ADMINISTRAÇÃO_SEMESTRE',
                    'TEORIA GERAL DA ADMINISTRAÇÃO_APROVADO',
                    'TEORIA GERAL DA ADMINISTRAÇÃO_REPROVADO',
                    'TEORIA GERAL DA ADMINISTRAÇÃO_FALTAS',
                    'TEORIA GERAL DA ADMINISTRAÇÃO_NOTA_1.0',
                    'TEORIA GERAL DA ADMINISTRAÇÃO_NOTA_2.0',
                    'TEORIA GERAL DA ADMINISTRAÇÃO_NOTA_3.0',
                    'TEORIA GERAL DA ADMINISTRAÇÃO_MEDIA_FINAL_1.0',
                    'TEORIA GERAL DA ADMINISTRAÇÃO_MEDIA_FINAL_2.0',
                    'PROGRAMAÇÃO',
                    'PROGRAMAÇÃO_SEMESTRE',
                    'PROGRAMAÇÃO_APROVADO',
                    'PROGRAMAÇÃO_REPROVADO',
                    'PROGRAMAÇÃO_FALTAS',
                    'PROGRAMAÇÃO_NOTA_1.0',
                    'PROGRAMAÇÃO_NOTA_2.0',
                    'PROGRAMAÇÃO_NOTA_3.0',
                    'PROGRAMAÇÃO_MEDIA_FINAL_1.0',
                    'PROGRAMAÇÃO_MEDIA_FINAL_2.0',
                    'CÁLCULO DIFERENCIAL E INTEGRAL',
                    'CÁLCULO DIFERENCIAL E INTEGRAL_SEMESTRE',
                    'CÁLCULO DIFERENCIAL E INTEGRAL_APROVADO',
                    'CÁLCULO DIFERENCIAL E INTEGRAL_REPROVADO',
                    'CÁLCULO DIFERENCIAL E INTEGRAL_FALTAS',
                    'CÁLCULO DIFERENCIAL E INTEGRAL_NOTA_1.0',
                    'CÁLCULO DIFERENCIAL E INTEGRAL_NOTA_2.0',
                    'CÁLCULO DIFERENCIAL E INTEGRAL_NOTA_3.0',
                    'CÁLCULO DIFERENCIAL E INTEGRAL_MEDIA_FINAL_1.0',
                    'CÁLCULO DIFERENCIAL E INTEGRAL_MEDIA_FINAL_2.0',
                    'TEORIA GERAL DOS SISTEMAS',
                    'TEORIA GERAL DOS SISTEMAS_SEMESTRE',
                    'TEORIA GERAL DOS SISTEMAS_APROVADO',
                    'TEORIA GERAL DOS SISTEMAS_REPROVADO',
                    'TEORIA GERAL DOS SISTEMAS_FALTAS',
                    'TEORIA GERAL DOS SISTEMAS_NOTA_1.0',
                    'TEORIA GERAL DOS SISTEMAS_NOTA_2.0',
                    'TEORIA GERAL DOS SISTEMAS_NOTA_3.0',
                    'TEORIA GERAL DOS SISTEMAS_MEDIA_FINAL_1.0',
                    'TEORIA GERAL DOS SISTEMAS_MEDIA_FINAL_2.0',
                    'PROGRAMAÇÃO ORIENTADA A OBJETOS I',
                    'PROGRAMAÇÃO ORIENTADA A OBJETOS I_SEMESTRE',
                    'PROGRAMAÇÃO ORIENTADA A OBJETOS I_APROVADO',
                    'PROGRAMAÇÃO ORIENTADA A OBJETOS I_REPROVADO',
                    'PROGRAMAÇÃO ORIENTADA A OBJETOS I_FALTAS',
                    'PROGRAMAÇÃO ORIENTADA A OBJETOS I_NOTA_1.0',
                    'PROGRAMAÇÃO ORIENTADA A OBJETOS I_NOTA_2.0',
                    'PROGRAMAÇÃO ORIENTADA A OBJETOS I_NOTA_3.0',
                    'PROGRAMAÇÃO ORIENTADA A OBJETOS I_MEDIA_FINAL_1.0',
                    'PROGRAMAÇÃO ORIENTADA A OBJETOS I_MEDIA_FINAL_2.0',
                    'ESTRUTURA DE DADOS',
                    'ESTRUTURA DE DADOS_SEMESTRE',
                    'ESTRUTURA DE DADOS_APROVADO',
                    'ESTRUTURA DE DADOS_REPROVADO',
                    'ESTRUTURA DE DADOS_FALTAS',
                    'ESTRUTURA DE DADOS_NOTA_1.0',
                    'ESTRUTURA DE DADOS_NOTA_2.0',
                    'ESTRUTURA DE DADOS_NOTA_3.0',
                    'ESTRUTURA DE DADOS_MEDIA_FINAL_1.0',
                    'ESTRUTURA DE DADOS_MEDIA_FINAL_2.0',
                    'ÁLGEBRA LINEAR',
                    'ÁLGEBRA LINEAR_SEMESTRE',
                    'ÁLGEBRA LINEAR_APROVADO',
                    'ÁLGEBRA LINEAR_REPROVADO',
                    'ÁLGEBRA LINEAR_FALTAS',
                    'ÁLGEBRA LINEAR_NOTA_1.0',
                    'ÁLGEBRA LINEAR_NOTA_2.0',
                    'ÁLGEBRA LINEAR_NOTA_3.0',
                    'ÁLGEBRA LINEAR_MEDIA_FINAL_1.0',
                    'ÁLGEBRA LINEAR_MEDIA_FINAL_2.0',
                    'ORGANIZAÇÃO, SISTEMAS E MÉTODOS',
                    'ORGANIZAÇÃO, SISTEMAS E MÉTODOS_SEMESTRE',
                    'ORGANIZAÇÃO, SISTEMAS E MÉTODOS_APROVADO',
                    'ORGANIZAÇÃO, SISTEMAS E MÉTODOS_REPROVADO',
                    'ORGANIZAÇÃO, SISTEMAS E MÉTODOS_FALTAS',
                    'ORGANIZAÇÃO, SISTEMAS E MÉTODOS_NOTA_1.0',
                    'ORGANIZAÇÃO, SISTEMAS E MÉTODOS_NOTA_2.0',
                    'ORGANIZAÇÃO, SISTEMAS E MÉTODOS_NOTA_3.0',
                    'ORGANIZAÇÃO, SISTEMAS E MÉTODOS_MEDIA_FINAL_1.0',
                    'ORGANIZAÇÃO, SISTEMAS E MÉTODOS_MEDIA_FINAL_2.0',
                    'FUNDAMENTOS DE SISTEMAS DE INFORMAÇÃO',
                    'FUNDAMENTOS DE SISTEMAS DE INFORMAÇÃO_SEMESTRE',
                    'FUNDAMENTOS DE SISTEMAS DE INFORMAÇÃO_APROVADO',
                    'FUNDAMENTOS DE SISTEMAS DE INFORMAÇÃO_REPROVADO',
                    'FUNDAMENTOS DE SISTEMAS DE INFORMAÇÃO_FALTAS',
                    'FUNDAMENTOS DE SISTEMAS DE INFORMAÇÃO_NOTA_1.0',
                    'FUNDAMENTOS DE SISTEMAS DE INFORMAÇÃO_NOTA_2.0',
                    'FUNDAMENTOS DE SISTEMAS DE INFORMAÇÃO_NOTA_3.0',
                    'FUNDAMENTOS DE SISTEMAS DE INFORMAÇÃO_MEDIA_FINAL_1.0',
                    'FUNDAMENTOS DE SISTEMAS DE INFORMAÇÃO_MEDIA_FINAL_2.0',
                    'PROGRAMAÇÃO WEB',
                    'PROGRAMAÇÃO WEB_SEMESTRE',
                    'PROGRAMAÇÃO WEB_APROVADO',
                    'PROGRAMAÇÃO WEB_REPROVADO',
                    'PROGRAMAÇÃO WEB_FALTAS',
                    'PROGRAMAÇÃO WEB_NOTA_1.0',
                    'PROGRAMAÇÃO WEB_NOTA_2.0',
                    'PROGRAMAÇÃO WEB_NOTA_3.0',
                    'PROGRAMAÇÃO WEB_MEDIA_FINAL_1.0',
                    'PROGRAMAÇÃO WEB_MEDIA_FINAL_2.0',
                    'ARQUITETURA DE COMPUTADORES',
                    'ARQUITETURA DE COMPUTADORES_SEMESTRE',
                    'ARQUITETURA DE COMPUTADORES_APROVADO',
                    'ARQUITETURA DE COMPUTADORES_REPROVADO',
                    'ARQUITETURA DE COMPUTADORES_FALTAS',
                    'ARQUITETURA DE COMPUTADORES_NOTA_1.0',
                    'ARQUITETURA DE COMPUTADORES_NOTA_2.0',
                    'ARQUITETURA DE COMPUTADORES_NOTA_3.0',
                    'ARQUITETURA DE COMPUTADORES_MEDIA_FINAL_1.0',
                    'ARQUITETURA DE COMPUTADORES_MEDIA_FINAL_2.0',
                    'PROBABILIDADE E ESTATÍSTICA',
                    'PROBABILIDADE E ESTATÍSTICA_SEMESTRE',
                    'PROBABILIDADE E ESTATÍSTICA_APROVADO',
                    'PROBABILIDADE E ESTATÍSTICA_REPROVADO',
                    'PROBABILIDADE E ESTATÍSTICA_FALTAS',
                    'PROBABILIDADE E ESTATÍSTICA_NOTA_1.0',
                    'PROBABILIDADE E ESTATÍSTICA_NOTA_2.0',
                    'PROBABILIDADE E ESTATÍSTICA_NOTA_3.0',
                    'PROBABILIDADE E ESTATÍSTICA_MEDIA_FINAL_1.0',
                    'PROBABILIDADE E ESTATÍSTICA_MEDIA_FINAL_2.0',
                    'BANCO DE DADOS',
                    'BANCO DE DADOS_SEMESTRE',
                    'BANCO DE DADOS_APROVADO',
                    'BANCO DE DADOS_REPROVADO',
                    'BANCO DE DADOS_FALTAS',
                    'BANCO DE DADOS_NOTA_1.0',
                    'BANCO DE DADOS_NOTA_2.0',
                    'BANCO DE DADOS_NOTA_3.0',
                    'BANCO DE DADOS_MEDIA_FINAL_1.0',
                    'BANCO DE DADOS_MEDIA_FINAL_2.0',
                    'ENGENHARIA DE SOFTWARE I',
                    'ENGENHARIA DE SOFTWARE I_SEMESTRE',
                    'ENGENHARIA DE SOFTWARE I_APROVADO',
                    'ENGENHARIA DE SOFTWARE I_REPROVADO',
                    'ENGENHARIA DE SOFTWARE I_FALTAS',
                    'ENGENHARIA DE SOFTWARE I_NOTA_1.0',
                    'ENGENHARIA DE SOFTWARE I_NOTA_2.0',
                    'ENGENHARIA DE SOFTWARE I_NOTA_3.0',
                    'ENGENHARIA DE SOFTWARE I_MEDIA_FINAL_1.0',
                    'ENGENHARIA DE SOFTWARE I_MEDIA_FINAL_2.0',
                    'PROGRAMAÇÃO ORIENTADA A OBJETOS II',
                    'PROGRAMAÇÃO ORIENTADA A OBJETOS II_SEMESTRE',
                    'PROGRAMAÇÃO ORIENTADA A OBJETOS II_APROVADO',
                    'PROGRAMAÇÃO ORIENTADA A OBJETOS II_REPROVADO',
                    'PROGRAMAÇÃO ORIENTADA A OBJETOS II_FALTAS',
                    'PROGRAMAÇÃO ORIENTADA A OBJETOS II_NOTA_1.0',
                    'PROGRAMAÇÃO ORIENTADA A OBJETOS II_NOTA_2.0',
                    'PROGRAMAÇÃO ORIENTADA A OBJETOS II_NOTA_3.0',
                    'PROGRAMAÇÃO ORIENTADA A OBJETOS II_MEDIA_FINAL_1.0',
                    'PROGRAMAÇÃO ORIENTADA A OBJETOS II_MEDIA_FINAL_2.0',
                    'SISTEMAS OPERACIONAIS',
                    'SISTEMAS OPERACIONAIS_SEMESTRE',
                    'SISTEMAS OPERACIONAIS_APROVADO',
                    'SISTEMAS OPERACIONAIS_REPROVADO',
                    'SISTEMAS OPERACIONAIS_FALTAS',
                    'SISTEMAS OPERACIONAIS_NOTA_1.0',
                    'SISTEMAS OPERACIONAIS_NOTA_2.0',
                    'SISTEMAS OPERACIONAIS_NOTA_3.0',
                    'SISTEMAS OPERACIONAIS_MEDIA_FINAL_1.0',
                    'SISTEMAS OPERACIONAIS_MEDIA_FINAL_2.0',
                    'PROJETO E ADMINISTRAÇÃO DE BANCO DE DADOS',
                    'PROJETO E ADMINISTRAÇÃO DE BANCO DE DADOS_SEMESTRE',
                    'PROJETO E ADMINISTRAÇÃO DE BANCO DE DADOS_APROVADO',
                    'PROJETO E ADMINISTRAÇÃO DE BANCO DE DADOS_REPROVADO',
                    'PROJETO E ADMINISTRAÇÃO DE BANCO DE DADOS_FALTAS',
                    'PROJETO E ADMINISTRAÇÃO DE BANCO DE DADOS_NOTA_1.0',
                    'PROJETO E ADMINISTRAÇÃO DE BANCO DE DADOS_NOTA_2.0',
                    'PROJETO E ADMINISTRAÇÃO DE BANCO DE DADOS_NOTA_3.0',
                    'PROJETO E ADMINISTRAÇÃO DE BANCO DE DADOS_MEDIA_FINAL_1.0',
                    'PROJETO E ADMINISTRAÇÃO DE BANCO DE DADOS_MEDIA_FINAL_2.0',
                    'ENGENHARIA DE SOFTWARE II',
                    'ENGENHARIA DE SOFTWARE II_SEMESTRE',
                    'ENGENHARIA DE SOFTWARE II_APROVADO',
                    'ENGENHARIA DE SOFTWARE II_REPROVADO',
                    'ENGENHARIA DE SOFTWARE II_FALTAS',
                    'ENGENHARIA DE SOFTWARE II_NOTA_1.0',
                    'ENGENHARIA DE SOFTWARE II_NOTA_2.0',
                    'ENGENHARIA DE SOFTWARE II_NOTA_3.0',
                    'ENGENHARIA DE SOFTWARE II_MEDIA_FINAL_1.0',
                    'ENGENHARIA DE SOFTWARE II_MEDIA_FINAL_2.0',
                    'REDES DE COMPUTADORES',
                    'REDES DE COMPUTADORES_SEMESTRE',
                    'REDES DE COMPUTADORES_APROVADO',
                    'REDES DE COMPUTADORES_REPROVADO',
                    'REDES DE COMPUTADORES_FALTAS',
                    'REDES DE COMPUTADORES_NOTA_1.0',
                    'REDES DE COMPUTADORES_NOTA_2.0',
                    'REDES DE COMPUTADORES_NOTA_3.0',
                    'REDES DE COMPUTADORES_MEDIA_FINAL_1.0',
                    'REDES DE COMPUTADORES_MEDIA_FINAL_2.0',
                    'EMPREENDEDORISMO EM INFORMÁTICA',
                    'EMPREENDEDORISMO EM INFORMÁTICA_SEMESTRE',
                    'EMPREENDEDORISMO EM INFORMÁTICA_APROVADO',
                    'EMPREENDEDORISMO EM INFORMÁTICA_REPROVADO',
                    'EMPREENDEDORISMO EM INFORMÁTICA_FALTAS',
                    'EMPREENDEDORISMO EM INFORMÁTICA_NOTA_1.0',
                    'EMPREENDEDORISMO EM INFORMÁTICA_NOTA_2.0',
                    'EMPREENDEDORISMO EM INFORMÁTICA_NOTA_3.0',
                    'EMPREENDEDORISMO EM INFORMÁTICA_MEDIA_FINAL_1.0',
                    'EMPREENDEDORISMO EM INFORMÁTICA_MEDIA_FINAL_2.0',
                    'GESTÃO DE PROJETO DE SOFTWARE',
                    'GESTÃO DE PROJETO DE SOFTWARE_SEMESTRE',
                    'GESTÃO DE PROJETO DE SOFTWARE_APROVADO',
                    'GESTÃO DE PROJETO DE SOFTWARE_REPROVADO',
                    'GESTÃO DE PROJETO DE SOFTWARE_FALTAS',
                    'GESTÃO DE PROJETO DE SOFTWARE_NOTA_1.0',
                    'GESTÃO DE PROJETO DE SOFTWARE_NOTA_2.0',
                    'GESTÃO DE PROJETO DE SOFTWARE_NOTA_3.0',
                    'GESTÃO DE PROJETO DE SOFTWARE_MEDIA_FINAL_1.0',
                    'GESTÃO DE PROJETO DE SOFTWARE_MEDIA_FINAL_2.0',
                    'PROGRAMAÇÃO VISUAL',
                    'PROGRAMAÇÃO VISUAL_SEMESTRE',
                    'PROGRAMAÇÃO VISUAL_APROVADO',
                    'PROGRAMAÇÃO VISUAL_REPROVADO',
                    'PROGRAMAÇÃO VISUAL_FALTAS',
                    'PROGRAMAÇÃO VISUAL_NOTA_1.0',
                    'PROGRAMAÇÃO VISUAL_NOTA_2.0',
                    'PROGRAMAÇÃO VISUAL_NOTA_3.0',
                    'PROGRAMAÇÃO VISUAL_MEDIA_FINAL_1.0',
                    'PROGRAMAÇÃO VISUAL_MEDIA_FINAL_2.0',
                    'MATEMÁTICA FINANCEIRA',
                    'MATEMÁTICA FINANCEIRA_SEMESTRE',
                    'MATEMÁTICA FINANCEIRA_APROVADO',
                    'MATEMÁTICA FINANCEIRA_REPROVADO',
                    'MATEMÁTICA FINANCEIRA_FALTAS',
                    'MATEMÁTICA FINANCEIRA_NOTA_1.0',
                    'MATEMÁTICA FINANCEIRA_NOTA_2.0',
                    'MATEMÁTICA FINANCEIRA_NOTA_3.0',
                    'MATEMÁTICA FINANCEIRA_MEDIA_FINAL_1.0',
                    'MATEMÁTICA FINANCEIRA_MEDIA_FINAL_2.0',
                    'SISTEMAS DE APOIO À DECISÃO',
                    'SISTEMAS DE APOIO À DECISÃO_SEMESTRE',
                    'SISTEMAS DE APOIO À DECISÃO_APROVADO',
                    'SISTEMAS DE APOIO À DECISÃO_REPROVADO',
                    'SISTEMAS DE APOIO À DECISÃO_FALTAS',
                    'SISTEMAS DE APOIO À DECISÃO_NOTA_1.0',
                    'SISTEMAS DE APOIO À DECISÃO_NOTA_2.0',
                    'SISTEMAS DE APOIO À DECISÃO_NOTA_3.0',
                    'SISTEMAS DE APOIO À DECISÃO_MEDIA_FINAL_1.0',
                    'SISTEMAS DE APOIO À DECISÃO_MEDIA_FINAL_2.0',
                    'ÉTICA',
                    'ÉTICA_SEMESTRE',
                    'ÉTICA_APROVADO',
                    'ÉTICA_REPROVADO',
                    'ÉTICA_FALTAS',
                    'ÉTICA_NOTA_1.0',
                    'ÉTICA_NOTA_2.0',
                    'ÉTICA_NOTA_3.0',
                    'ÉTICA_MEDIA_FINAL_1.0',
                    'ÉTICA_MEDIA_FINAL_2.0',
                    #'tempo_relativo',
                    'status'
                      ]
df_merged_7 = df_merged_7[colunas_ordenadas]"""

KeyError: "['ultimo_periodo', 'ch_cumprida', 'ch_cumprida_dividida', 'ch_reprovacao', 'taxa_reprovacao', 'semestre_total', 'semestre_dividido', 'matricula'] not in index"

In [ ]:
status_data = df_merged_7['status'].value_counts()
status_data 

status
-1    410
 1    270
Name: count, dtype: int64

# Salvando DataFrames

Para salvar o DataFrame no formato CSV com a separação por ',' e garantir que os dados numéricos estejam no formato correto.

In [ ]:
df_merged_7.to_csv('df_bsi.csv', index=False, sep=',', quoting=csv.QUOTE_NONNUMERIC)

In [ ]:
df_merged_7

,discente,ano_ingresso,ultimo_periodo,ch_cumprida,ch_cumprida_dividida,ch_reprovacao,taxa_reprovacao,semestre_total,semestre_dividido,matricula,ALGORITMOS E LÓGICA DE PROGRAMAÇÃO,ALGORITMOS E LÓGICA DE PROGRAMAÇÃO_SEMESTRE,ALGORITMOS E LÓGICA DE PROGRAMAÇÃO_APROVADO,ALGORITMOS E LÓGICA DE PROGRAMAÇÃO_REPROVADO,ALGORITMOS E LÓGICA DE PROGRAMAÇÃO_FALTAS,ALGORITMOS E LÓGICA DE PROGRAMAÇÃO_NOTA_1.0,ALGORITMOS E LÓGICA DE PROGRAMAÇÃO_NOTA_2.0,ALGORITMOS E LÓGICA DE PROGRAMAÇÃO_NOTA_3.0,ALGORITMOS E LÓGICA DE PROGRAMAÇÃO_MEDIA_FINAL_1.0,ALGORITMOS E LÓGICA DE PROGRAMAÇÃO_MEDIA_FINAL_2.0,INTRODUÇÃO À INFORMÁTICA,INTRODUÇÃO À INFORMÁTICA_SEMESTRE,INTRODUÇÃO À INFORMÁTICA_APROVADO,INTRODUÇÃO À INFORMÁTICA_REPROVADO,INTRODUÇÃO À INFORMÁTICA_FALTAS,INTRODUÇÃO À INFORMÁTICA_NOTA_1.0,INTRODUÇÃO À INFORMÁTICA_NOTA_2.0,INTRODUÇÃO À INFORMÁTICA_NOTA_3.0,INTRODUÇÃO À INFORMÁTICA_MEDIA_FINAL_1.0,INTRODUÇÃO À INFORMÁTICA_MEDIA_FINAL_2.0,FUNDAMENTOS DE MATEMÁTICA,FUNDAMENTOS DE MATEMÁTICA_SEMESTRE,FUNDAMENTOS DE MATEMÁTICA_APROVADO,FUNDAMENTOS DE MATEMÁTICA_REPROVADO,FUNDAMENTOS DE MATEMÁTICA_FALTAS,FUNDAMENTOS DE MATEMÁTICA_NOTA_1.0,FUNDAMENTOS DE MATEMÁTICA_NOTA_2.0,FUNDAMENTOS DE MATEMÁTICA_NOTA_3.0,FUNDAMENTOS DE MATEMÁTICA_MEDIA_FINAL_1.0,FUNDAMENTOS DE MATEMÁTICA_MEDIA_FINAL_2.0,LÓGICA,LÓGICA_SEMESTRE,LÓGICA_APROVADO,LÓGICA_REPROVADO,LÓGICA_FALTAS,LÓGICA_NOTA_1.0,LÓGICA_NOTA_2.0,LÓGICA_NOTA_3.0,LÓGICA_MEDIA_FINAL_1.0,LÓGICA_MEDIA_FINAL_2.0,TEORIA GERAL DA ADMINISTRAÇÃO,TEORIA GERAL DA ADMINISTRAÇÃO_SEMESTRE,TEORIA GERAL DA ADMINISTRAÇÃO_APROVADO,TEORIA GERAL DA ADMINISTRAÇÃO_REPROVADO,TEORIA GERAL DA ADMINISTRAÇÃO_FALTAS,TEORIA GERAL DA ADMINISTRAÇÃO_NOTA_1.0,TEORIA GERAL DA ADMINISTRAÇÃO_NOTA_2.0,TEORIA GERAL DA ADMINISTRAÇÃO_NOTA_3.0,TEORIA GERAL DA ADMINISTRAÇÃO_MEDIA_FINAL_1.0,TEORIA GERAL DA ADMINISTRAÇÃO_MEDIA_FINAL_2.0,PROGRAMAÇÃO,PROGRAMAÇÃO_SEMESTRE,PROGRAMAÇÃO_APROVADO,PROGRAMAÇÃO_REPROVADO,PROGRAMAÇÃO_FALTAS,PROGRAMAÇÃO_NOTA_1.0,PROGRAMAÇÃO_NOTA_2.0,PROGRAMAÇÃO_NOTA_3.0,PROGRAMAÇÃO_MEDIA_FINAL_1.0,PROGRAMAÇÃO_MEDIA_FINAL_2.0,CÁLCULO DIFERENCIAL E INTEGRAL,CÁLCULO DIFERENCIAL E INTEGRAL_SEMESTRE,CÁLCULO DIFERENCIAL E INTEGRAL_APROVADO,CÁLCULO DIFERENCIAL E INTEGRAL_REPROVADO,CÁLCULO DIFERENCIAL E INTEGRAL_FALTAS,CÁLCULO DIFERENCIAL E INTEGRAL_NOTA_1.0,CÁLCULO DIFERENCIAL E INTEGRAL_NOTA_2.0,CÁLCULO DIFERENCIAL E INTEGRAL_NOTA_3.0,CÁLCULO DIFERENCIAL E INTEGRAL_MEDIA_FINAL_1.0,CÁLCULO DIFERENCIAL E INTEGRAL_MEDIA_FINAL_2.0,TEORIA GERAL DOS SISTEMAS,TEORIA GERAL DOS SISTEMAS_SEMESTRE,TEORIA GERAL DOS SISTEMAS_APROVADO,TEORIA GERAL DOS SISTEMAS_REPROVADO,TEORIA GERAL DOS SISTEMAS_FALTAS,TEORIA GERAL DOS SISTEMAS_NOTA_1.0,TEORIA GERAL DOS SISTEMAS_NOTA_2.0,TEORIA GERAL DOS SISTEMAS_NOTA_3.0,TEORIA GERAL DOS SISTEMAS_MEDIA_FINAL_1.0,TEORIA GERAL DOS SISTEMAS_MEDIA_FINAL_2.0,PROGRAMAÇÃO ORIENTADA A OBJETOS I,PROGRAMAÇÃO ORIENTADA A OBJETOS I_SEMESTRE,PROGRAMAÇÃO ORIENTADA A OBJETOS I_APROVADO,PROGRAMAÇÃO ORIENTADA A OBJETOS I_REPROVADO,PROGRAMAÇÃO ORIENTADA A OBJETOS I_FALTAS,PROGRAMAÇÃO ORIENTADA A OBJETOS I_NOTA_1.0,PROGRAMAÇÃO ORIENTADA A OBJETOS I_NOTA_2.0,PROGRAMAÇÃO ORIENTADA A OBJETOS I_NOTA_3.0,PROGRAMAÇÃO ORIENTADA A OBJETOS I_MEDIA_FINAL_1.0,PROGRAMAÇÃO ORIENTADA A OBJETOS I_MEDIA_FINAL_2.0,ESTRUTURA DE DADOS,ESTRUTURA DE DADOS_SEMESTRE,ESTRUTURA DE DADOS_APROVADO,ESTRUTURA DE DADOS_REPROVADO,ESTRUTURA DE DADOS_FALTAS,ESTRUTURA DE DADOS_NOTA_1.0,ESTRUTURA DE DADOS_NOTA_2.0,ESTRUTURA DE DADOS_NOTA_3.0,ESTRUTURA DE DADOS_MEDIA_FINAL_1.0,ESTRUTURA DE DADOS_MEDIA_FINAL_2.0,ÁLGEBRA LINEAR,ÁLGEBRA LINEAR_SEMESTRE,ÁLGEBRA LINEAR_APROVADO,ÁLGEBRA LINEAR_REPROVADO,ÁLGEBRA LINEAR_FALTAS,ÁLGEBRA LINEAR_NOTA_1.0,ÁLGEBRA LINEAR_NOTA_2.0,ÁLGEBRA LINEAR_NOTA_3.0,ÁLGEBRA LINEAR_MEDIA_FINAL_1.0,ÁLGEBRA LINEAR_MEDIA_FINAL_2.0,"ORGANIZAÇÃO, SISTEMAS E MÉTODOS","ORGANIZAÇÃO, SISTEMAS E MÉTODOS_SEMESTRE","ORGANIZAÇÃO, SISTEMAS E MÉTODOS_APROVADO","ORGANIZAÇÃO, SISTEMAS E MÉTODOS_REPROVADO","ORGANIZAÇÃO, SISTEMAS E MÉTODOS_FALTAS","ORGANIZAÇÃO, SIS

In [ ]:
df_dados_filtrado.to_csv('df_filtrado.csv', index=False, sep=',', quoting=csv.QUOTE_NONNUMERIC)

In [ ]:
tabela_final.to_csv('tabela_final.csv', index=False, sep=',', quoting=csv.QUOTE_NONNUMERIC)